<h1 style="color:orange"> PARTE I </h1>

In [1]:
!pip3 install xlsxwriter

In [2]:
import json                                                 
import pandas as pd                                         
import numpy as np
import requests
from bs4 import BeautifulSoup
import string
from nltk.tokenize import RegexpTokenizer
from nltk import bigrams
from nltk.util import ngrams
from nltk import ngrams
import collections
import re
import os
import urllib.request
from nltk.corpus import stopwords
import nltk
from matplotlib.pyplot import figure
pd.options.display.max_columns = None
import pandas.io.formats.excel
pandas.io.formats.excel.header_style = None
import datetime
BOLD = '\033[1m'
END = '\033[0m'
nltk.download('stopwords')
from pandas.io import gbq
import xlsxwriter

[nltk_data] Downloading package stopwords to C:\Users\Dell
[nltk_data]     User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Define Functions

## Time Stamp Function

In [3]:
def get_timestampYMD():
    return datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

## Upload Stopwords

In [4]:
sw_es=stopwords.words('spanish')
sw_pt=stopwords.words('portuguese')
sw_en=stopwords.words('english')
sw_es_title=[word.title() for word in sw_es]
sw_pt_title=[word.title() for word in sw_pt]
sw_en_title=[word.title() for word in sw_en]

## Extract Uppercase Words

In [5]:
def get_mayusculas(texto):
    texto=str(texto)
    palabras_importantes=[]
    mayusculas=(r"([A-Z][a-zÁ-ÿ0-9]{1,20}\s?\,?\.?\s?)")
    texto=re.sub('[^\w\s]',' ',texto)
    texto=re.sub('[0-9]+', '', texto)  
    tokenizer=RegexpTokenizer(r'\w+')
    texto=tokenizer.tokenize(texto)
    texto=[word for word in texto if word not in sw_pt]
    texto=[word for word in texto if word not in sw_es]
    texto=[word for word in texto if word not in sw_en]
    texto=[word for word in texto if word not in sw_pt_title]
    texto=[word for word in texto if word not in sw_es_title]
    texto=[word for word in texto if word not in sw_en_title]
    tokens=[word.strip() for word in texto if word is not None]
    tokens=[word.strip() for word in tokens if len(word)>1]
    palabras_importantes=[word for word in tokens if word.istitle()]
    return palabras_importantes

## Adjust Country & Subsidiary

In [6]:
def country_sub(country):
    if country =='Panama' or country =='PA' or country=='panama' or country=='Panamá':
        return ['Panama','Rola']
    elif country =='Costa Rica' or country =='CR' or country=='costa rica':
        return ['Costa Rica','Rola']
    elif country =='Honduras' or country =='HN' or country=='honduras':
        return ['Honduras','Rola']
    elif country =='Nicaragua' or country =='NI' or country=='nicaragua':
        return ['Nicaragua','Rola']
    elif country =='Guatemala' or country =='GT' or country=='guatemala':
        return ['Guatemala','Rola']
    elif country =='El Salvador' or country =='SV' or country=='el salvador':
        return ['El Salvador','Rola']
    elif country =='Venezuela' or country =='VE' or country=='venezuela':
        return ['Venezuela','Rola']
    elif country =='Puerto Rico' or country =='PR' or country=='puerto rico':
        return ['Puerto Rico','Rola']
    elif country =='Dominican Republic' or country =='DO' or country=='dominican republic' or country=='republica dominicana' or country=='Rep. Dominicana':
        return ['Dominican Republic','Rola']
    elif country =='Trinidad & Tobago' or country =='TT' or country=='trinidad & tobago' or country=='Trinidad y Tobago' or country=='T&T':
        return ['Trinidad & Tobago','Rola']
    elif country =='Jamaica' or country =='JM' or country=='jamaica':
        return ['Jamaica','Rola']
    elif country =='Peru' or country =='PE' or country=='peru' or country=='Perú':
        return ['Peru','Rola']
    elif country =='Ecuador' or country =='EC' or country=='ecuador' or country=='equador' or country=='EQUADOR':
        return ['Ecuador','Rola']
    elif country =='Bolivia' or country =='BO' or country=='bolivia':
        return ['Bolivia','Rola']
    elif country =='Paraguay' or country =='PY' or country=='paraguay' or country=='paraguai' or country=='Paraguai':
        return ['Paraguay','Rola']
    elif country =='Uruguay' or country =='UY' or country=='uruguay'or country=='uruguai' or country=='Uruguai':
        return ['Uruguay','Rola']
    elif country =='Mexico' or country =='MX' or country=='mexico' or country=='México':
        return ['Mexico','Mexico']
    elif country =='Argentina' or country =='AR' or country=='argentina':
        return ['Argentina','Argentina']
    elif country =='Brazil' or country =='BR' or country=='brazil' or country=='brasil' or country=='Brasil':
        return ['Brazil','Brazil']
    elif country =='Chile' or country =='CL' or country=='chile':
        return ['Chile','Chile']
    elif country =='Colombia' or country =='CL' or country=='colombia':
        return ['Colombia','Colombia']
    else:
        return ['Revisar','Revisar']

## Adjust Month & Quarters

In [7]:
def find_quarters(month):
    if month =='1': 
        return ['January','01','Q1','H1']
    elif month =='2': 
        return ['February','02','Q1','H1']
    elif month =='3': 
        return ['March','03','Q1','H1']
    elif month =='4': 
        return ['April','04','Q2','H1']
    elif month =='5': 
        return ['May','05','Q2','H1']
    elif month =='6': 
        return ['June','06','Q2','H1']
    elif month =='7': 
        return ['July','07','Q3','H2']
    elif month =='8': 
        return ['August','08','Q3','H2']
    elif month =='9': 
        return ['September','09','Q3','H2']
    elif month =='10': 
        return ['October','10','Q4','H2']
    elif month =='11': 
        return ['November','11','Q4','H2']
    elif month =='12': 
        return ['December','12','Q4','H2']

## Remove Punctuation Marks

In [8]:
def remove_punct(texto):
    try:
        texto=texto.replace(".",' ').replace(";",' ').replace(":",' ').replace(",",' ')
        texto=texto.replace("(",' ').replace(")",' ').replace("|",' ').replace('"',' ')
        texto=texto.replace("%",' ').replace("$",' ').replace("/",' ').replace('\'',' ')
        texto=texto.replace("-",' ').replace("_",' ').replace("*",' ').replace('+',' ')
        texto=texto.replace("#",' ').replace("@",' ').replace("!",' ').replace('?',' ')
    except:
        pass
    return texto 

## Clean Text - Contents

In [9]:
def clean_text(texto):
    texto=texto.lower()
    texto=remove_punct(texto)
    return texto

## Tokenize Text

In [10]:
def tokenizar(texto):
    tokens = [t for t in texto.split()]
    return tokens

## Create Keywords Dictionary

In [11]:
def crear_dicc_keywords(df_keywords):
    df_keywords=df_keywords.fillna('exxxtract')
    area_dict = df_keywords.to_dict('list')
    for k,v in area_dict.items():
        nv=list(set(v))
        nv=[x for x in v if x != 'exxxtract']
        nv=list(set(nv))
        area_dict[k]=nv
    return area_dict

## Convert Finded Words To a List

In [12]:
def convert_to_words(list):
    new_list=[]
    for item,item2 in zip(list,category_list_found):
        if int(item)>0: 
            new_list.append(item2.replace('_FOUND',''))
    return new_list

## Read '.JSON' Files

In [13]:
def read_json(file_path):
    data={}
    with open(file_path,encoding="utf8") as json_file:
        arq = json.load(json_file)
    data['Notas']=arq
    return data

## Create The Required DataFrame To Allocate Notes

In [14]:
def crear_df_json(data):                                                                                  
    df_n=pd.DataFrame(columns=['IdNoticia', 'Fecha', 'Hora', 'TipoDeMedio', 'Medio', 'País', 'Sección','Título', 'Cuerpo', 'Tier',
                               'NroCaracteres', 'Tono', 'LinkImagen','CPE', 'Moneda', 'Audiencia', 'Tema', 'Empresa', 'NroPagina',
                               'Dimensión', 'CirculacionMedio', 'AutorConductor', 'ResumenAclaracion','LinkNota'])
    for item in data['Notas']:                                                                            
        dftemp=pd.DataFrame.from_dict(item,orient='index')                                                
        dftemp=dftemp.T                                                                                   
        df_n=df_n.append(dftemp)                                                                          
    df_n=df_n.reset_index()                                                                               
    return df_n

## Find Words In a Text - Contents

In [15]:
def find_single_word_in_tokenized_text(row,keyword):
    if len([x for x in row if x ==keyword])>=1: 
        return keyword 
    else:
        return

## Categorize Notes

In [16]:
def categoriza_nota(Brands,Priority_products,text):
    text=str(text)
    text=text.replace('[','').replace(']','')
    menciones=re.findall(Brands,text)
    if menciones:
        dicc_menciones={}
        dicc_menciones[list(set(menciones))[0]]=len(menciones)
    else:
        dicc_menciones={}
    productos_enc=[]
    for item in Priority_products:
        if item:
            p_enc=re.findall(item,text)
            productos_enc.append(p_enc)
        else:
            None
    dicc={}
    for item2 in productos_enc:
        if item2:
            dicc[list(set(item2))[0]]=len(item2)
        else:
            None
    flat_products = [x for sublist in productos_enc for x in sublist]
    cuenta=len(menciones)+len(flat_products)
    if cuenta>=3:
        cat_nota='Prominent'
    elif cuenta>1 and cuenta<3:
        cat_nota='Relevant'
    elif cuenta==1:
        cat_nota='Passive'
    else:
        cat_nota='Non_related'
    return dicc_menciones,dicc,cat_nota

# Mas de 3 menciones de producto+marca = Prominent
# 1<mención de Marca + Producto<3 = Relevant
# 1=Pasive
# 0=Null

## Get Full Content Using Urlib Library

In [17]:
import urllib.request
import random


user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]
def get_full_content(url):
    for i in range(1,6):
    #Pick a random user agent
        user_agent = random.choice(user_agent_list)
        #Set the headers 
        headers = {'User-Agent': user_agent}

        if url:
            try:
                req = urllib.request.Request(url,headers={'User-Agent': user_agent})
                response = urllib.request.urlopen(req)
                html = response.read()
 
                soup = BeautifulSoup(html)
                paragraphs=re.findall(r'<p>(.*?)</p>',str(soup))
                paragraphs=re.sub(r'<a href=.+?(?=)>|</a>|\xa0|<strong>|</strong>|<i(.*?)</i>|<img(.*?)>','',str(paragraphs))
                paragraphs2=''.join(paragraphs)
                paragraphs2=paragraphs2.replace('\n','').replace('\t','').replace('\r','')
            except:
                paragraphs2='Sin Informacion'
                print('ups')

    return(paragraphs2)

## Clean & Tokenize Text

In [18]:
def clean_and_tokenize(texto):
    texto=str(texto).lower()
    texto=re.sub('[^\w\s]',' ',texto)
    #texto=re.sub('[0-9]+', '', texto)  
    tokenizer=RegexpTokenizer(r'\w+')
    #print(tokenizer)
    texto=tokenizer.tokenize(texto)
    #print(texto)
    texto=[word for word in texto if word not in sw_pt]
    texto=[word for word in texto if word not in sw_es]
    texto=[word for word in texto if word not in swsp]
    tokens=[word.strip() for word in texto if word is not None]
    tokens=[word.strip() for word in tokens if len(word)>1]
    return tokens

## Find Most Common Word

In [19]:
def find_most_common(texto,filtradas,qty):
    tokens=clean_and_tokenize(texto)
    tokens2=[word for word in tokens if word not in filtradas]
    bigrm = list(nltk.bigrams(tokens2))
    found=collections.Counter(bigrm)
    tops=found.most_common(qty)
    mf=[]
    for i in range(0,len(tops)):
        terms=tops[i][0]
        freq=tops[i][1]
        most_frequent=(terms,freq)
        mf.append(most_frequent)
    return mf

## Find Nokia Words - Batch 1

In [20]:
def MC_WORDS(df,filtro_company,qty):
    filtradas=[]
    df['Contents_MC']=df['Contents'].apply(lambda row : str(row).replace('http://','').replace('https://','').replace('www',''))
    for filtro in filtro_company:
        df['Contents_MC']=df['Contents'].apply(lambda row : str(row).replace(filtro,''))
    lista02=df['Contents'].tolist()
    mc03=find_most_common(lista02,filtradas,qty)
    return mc03

## Find Nokia Words - Batch 2

In [21]:
def MC_WORDS2(df,filtro_company,qty):
    filtradas=[]
    df['Full Text_MC']=df['Full Text'].apply(lambda row : str(row).replace('http://','').replace('https://','').replace('www',''))
    for filtro in swsp:
        df['Full Text_MC']=df['Full Text'].apply(lambda row : str(row).replace(filtro,''))
    lista02=df['Full Text_MC'].tolist()
    mc03=find_most_common(lista02,filtradas,qty)
    return mc03

## Define Additional Spanish Stopwords

In [22]:
swsp=['de','la','que','el','en','y','a','los','del','se','las','por','un','para','con','no',
      'una','su','al','lo','como','más','pero','sus','le','ya','o','este','sí','porque','esta',
      'entre','cuando','muy','sin','sobre','también','me','hasta','hay','donde','quien','desde',
      'todo','nos','durante','todos','uno','les','ni','contra','otros','ese','eso','ante','ellos',
      'e','esto','mí','antes','algunos','qué','unos','yo','otro','otras','otra','él','tanto','esa',
      'estos','mucho','quienes','nada','muchos','cual','poco','ella','estar','estas','algunas','algo',
      'nosotros','mi','mis','tú','te','ti','tu','tus','ellas','nosotras','vosostros','vosostras','os',
      'mío','mía','míos','mías','tuyo','tuya','tuyos','tuyas','suyo','suya','suyos','suyas','nuestro',
      'nuestra','nuestros','nuestras','vuestro','vuestra','vuestros','vuestras','esos','esas','estoy',
      'estás','está','estamos','estáis','están','esté','estés','estemos','estéis','estén','estaré',
      'estarás','estará','estaremos','estaréis','estarán','estaría','estarías','estaríamos','estaríais',
      'estarían','estaba','estabas','estábamos','estabais','estaban','estuve','estuviste','estuvo',
      'estuvimos','estuvisteis','estuvieron','estuviera','estuvieras','estuviéramos','estuvierais',
      'estuvieran','estuviese','estuvieses','estuviésemos','estuvieseis','estuviesen','estando','estado',
      'estada','estados','estadas','estad','he','has','ha','hemos','habéis','han','haya','hayas',
      'hayamos','hayáis','hayan','habré','habrás','habrá','habremos','habréis','habrán','habría',
      'habrías','habríamos','habríais','habrían','había','habías','habíamos','habíais','habían','hube',
      'hubiste','hubo','hubimos','hubisteis','hubieron','hubiera','hubieras','hubiéramos','hubierais',
      'hubieran','hubiese','hubieses','hubiésemos','hubieseis','hubiesen','habiendo','habido','habida',
      'habidos','habidas','soy','eres','es','somos','sois','son','sea','seas','seamos','seáis','sean',
      'seré','serás','será','seremos','seréis','serán','sería','serías','seríamos','seríais','serían',
      'era','eras','éramos','erais','eran','fui','fuiste','fue','fuimos','fuisteis','fueron','fuera',
      'fueras','fuéramos','fuerais','fueran','fuese','fueses','fuésemos','fueseis','fuesen','sintiendo',
      'sentido','sentida','sentidos','sentidas','siente','sentid','tengo','tienes','tiene','tenemos',
      'tenéis','tienen','tenga','tengas','tengamos','tengáis','tengan','tendré','tendrás','tendrá',
      'tendremos','tendréis','tendrán','tendría','tendrías','tendríamos','tendríais','tendrían','tenía',
      'tenías','teníamos','teníais','tenían','tuve','tuviste','tuvo','tuvimos','tuvisteis','tuvieron',
      'tuviera','tuvieras','tuviéramos','tuvierais','tuvieran','tuviese','tuvieses','tuviésemos',
      'tuvieseis','tuviesen','teniendo','tenido','tenida','tenidos','tenidas','tened','https','co','mucha','rt','poner',
      'interlocutor','interlocutora','presidente','AMLO','Andrés Manuel López Obrador','Andres Manuel Lopez Obrador',
      'Andres Manuel','Lopez Obrador','Andrés Manuel','López Obrador','PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR',
      'INTERLOCUTORA','INTERLOCUTOR','andrés','manuel','lópez','obrador','interlocutor','interlocutora','br']

## Remove Additional Spanish Stopwords

In [23]:
def remove_swsp(texto):                    
    tokens = [t for t in texto.split()]
    clean_tokens = tokens[:]
    for token in tokens:
        if token in swsp:
            clean_tokens.remove(token)
    return clean_tokens

## Clean & Tokenize Text

In [24]:
def clean_text_wt(texto):
    clean_tokens=clean_and_tokenize(texto)
    texto = ' '.join(clean_tokens)
    return texto

## Split Text In N-Grams

In [25]:
def divide_en_ngramas(texto,n):
    if len(texto)>0:
        n_grams=[]
        enegramas = ngrams(texto.split(), n)
        for grams in enegramas:
            i=0
            division=''
            while i<n: 
                division=division+' '+grams[i]
                i+=1
            n_grams.append(division.lstrip())
    else:
        return ''
    return n_grams

## Detect N-Grams In Text

In [26]:
def detect_ngram_in_text(texto,keyword):
    tam_keyword=len(keyword.split(' '))
    if tam_keyword==1:

        texto_fuente=clean_and_tokenize(texto)
        if keyword in texto_fuente:
            return 1
        else: 
            return 0
    else:
        texto_fuente=divide_en_ngramas(texto,tam_keyword)
        if keyword in texto_fuente:
            return 1
        else: 
            return 0

## Detect N-Grams In Text

In [27]:
def detect_ngram_in_text2(texto,keyword):
    tam_keyword=len(keyword.split(' '))
    if tam_keyword==1:

        texto_fuente=clean_and_tokenize(texto)
        if keyword in texto_fuente:
            return 1
        else: 
            return 0
    else:
        texto_fuente=divide_en_ngramas(texto,tam_keyword)
        if keyword in texto_fuente:
            return 1
        else: 
            return 0

## Find Keywords Matches

In [28]:
def encuentra_matches(row,keyword):
    texto=str(row)
    n=len(keyword.split(' '))
    f=divide_en_ngramas(texto,n)
    if n>0:
        matched=process.extract(keyword,f)
        best_options=[]
        for item in matched:
            if item[1]>=91:
                best_options.append(item[0])
    else:
        return 'N/A'
    print(matched)
    print(best_options)
    return best_options

## Find Keywords Matches

In [29]:
def encuentra_matches(row,keyword):
    row=str(row)
    matched=process.extract(keyword,row.split(' '))
    best_options=[]
    for item in matched:
        if item[1]>=91:
            best_options.append(item[0])
    return best_options

## Adjust Required 'Exchange Rate' To USA Dollars

In [30]:
def cambio_moneda(mor,cpe):
    if mor=='Peso Argentino':
        return format(cpe/ARS,'.3f')
    elif mor=='Peso Uruguayo':
        return format(cpe/UYU,'.3f')
    elif mor=='Peso Mejicano':
        return format(cpe/MXN,'.3f')
    elif mor=='Peso Colombiano':
        return format(cpe/COP,'.3f')
    elif mor=='Nuevo Sol Peruano':
        return format(cpe/PEN,'.3f')
    elif mor=='Guaraní Paraguayo':
        return format(cpe/PYG,'.3f')
    elif mor=='Peso Chileno':
        return format(cpe/CLP,'.3f')
    elif mor=='Lempira Hondureña':
        return format(cpe/HNL,'.3f')
    elif mor=='Dólar Estadounidense':
        return format(cpe/1,'.3f')
    elif mor=='Quetzal Guatemalteco':
        return format(cpe/GTQ,'.3f')
    elif mor=='Boliviano':
        return format(cpe/BOB,'.3f')
    elif mor=='Colón Costarricense':
        return format(cpe/CRC,'.3f')
    else: return cpe

## Read Source Data To Get 'Exchange Rate' - USA Dollar

In [31]:
tbl_monedas = pd.read_html('https://www.exchange-rates.org/currentRates/A/USD')
tbl_monedas[1]
ARS=tbl_monedas[1]['Exchange Rate'][0]
float(ARS)
BSD=tbl_monedas[1]['Exchange Rate'][1]
float(BSD)
BBD=tbl_monedas[1]['Exchange Rate'][2]
float(BBD)
BZD=tbl_monedas[1]['Exchange Rate'][3]
float(BZD)
BMD=tbl_monedas[1]['Exchange Rate'][4]
float(BMD)
BOB=tbl_monedas[1]['Exchange Rate'][5]
float(BOB)
BRL=tbl_monedas[1]['Exchange Rate'][6]
float(BRL)
CAD=tbl_monedas[1]['Exchange Rate'][7]
float(CAD)
KYD=tbl_monedas[1]['Exchange Rate'][8]
#float(KYD)
CLP=tbl_monedas[1]['Exchange Rate'][9]
float(CLP)
COP=tbl_monedas[1]['Exchange Rate'][10]
float(COP)
CRC=tbl_monedas[1]['Exchange Rate'][11]
float(CRC)
CUP=tbl_monedas[1]['Exchange Rate'][12]
float(CUP)
DOP=tbl_monedas[1]['Exchange Rate'][13]
float(DOP)
XCD=tbl_monedas[1]['Exchange Rate'][14]
float(XCD)
GTQ=tbl_monedas[1]['Exchange Rate'][15]
float(GTQ)
HTG=tbl_monedas[1]['Exchange Rate'][16]
float(HTG)
HNL=tbl_monedas[1]['Exchange Rate'][17]
float(HNL)
JMD=tbl_monedas[1]['Exchange Rate'][18]
float(JMD)
MXN=tbl_monedas[1]['Exchange Rate'][19]
#(MXN).astype(float)
ANG=tbl_monedas[1]['Exchange Rate'][20]
float(ANG)
NIO=tbl_monedas[1]['Exchange Rate'][21]
float(NIO)
PAB=tbl_monedas[1]['Exchange Rate'][22]
float(PAB)
PYG=tbl_monedas[1]['Exchange Rate'][23]
float(PYG)
PEN=tbl_monedas[1]['Exchange Rate'][24]
float(PEN)
TTD=tbl_monedas[1]['Exchange Rate'][25]
float(TTD)
UYU=tbl_monedas[1]['Exchange Rate'][26]
float(UYU)
VES=tbl_monedas[1]['Exchange Rate'][27]
float(VES)
print(ARS)

97.78661


In [32]:
ARS

97.78661

In [33]:
type(ARS)

numpy.float64

## Create Empty Required DataFrame

In [34]:
ANALIZED=pd.DataFrame(columns=['No','Source','Host','Link', 'Link Image',
                    'Date(ET)','Full Month','Month','Q','H','Author Name','Language','Country','Sub','Title','Contents', 
                    'Impact','Topic','Action','Category','Comments','Resumen_Categorias','Mayusculas_Title','Mayusculas_Contents',
'Nokia Sentiment',
 'HMD Sentiment',
 'Samsung Sentiment',
 'Motorola Sentiment',
 'Huawei Sentiment',
 'Xiaomi Sentiment',
 'LG Sentiment'                         
])
ANALIZED

,No,Source,Host,Link,Link Image,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment


## Read MediaList Excel File

In [35]:
medialist=pd.read_excel('/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/SOURCE FILES/Nokia_Media List.xlsx')

medialist.fillna('No Data')

medialist['MediaList']='Yes'

medialist['clean_link_site']=medialist['Link/Site'].apply(lambda row : str(row).replace('http://','').replace('https://','').replace('/',''))

medialist=medialist[['Publication','Host','Tier', 'Media Type','clean_link_site', 'MediaList']]
medialist.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/SOURCE FILES/Nokia_Media List.xlsx'

In [45]:
from zipfile import *
import glob
zip_files = glob.glob('/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/ZIP FILES/*.zip')
unzipped_files='/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1'
for zip_file in zip_files:
   file_name = zip_file.split('\\')[-1]
   print('Processing file:', file_name)
   name_unzipped=file_name.replace('.zip','.json')
   zipdata = ZipFile(zip_file)
   zipinfos = zipdata.infolist()
   for zipinfo in zipinfos:
     zipinfo.filename =name_unzipped
     zipdata.extract(zipinfo,unzipped_files)
     print('Extracted to unzipped folder as: ',name_unzipped)

Processing file: /home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/ZIP FILES/NokiaFile-20210629121553773.zip
Extracted to unzipped folder as:  /home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/ZIP FILES/NokiaFile-20210629121553773.json
Processing file: /home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/ZIP FILES/NokiaFile-20210707121611128.zip
Extracted to unzipped folder as:  /home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/ZIP FILES/NokiaFile-20210707121611128.json
Processing file: /home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/ZIP FILES/NokiaFile-20210716121545303.zip
Extracted to unzipped folder as:  /home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/ZIP FILES/NokiaFile-20210716121545303.json
Processing file: /home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOM

## Build a Required Empty DataFrame

In [41]:
# Build Required DataFrame (According To JSON File Format From Global News)
df_notas=pd.DataFrame(columns=['IdNoticia', 'Fecha', 'Hora', 'TipoDeMedio', 'Medio', 'País',
       'Programa - Sección', 'Título', 'Tier', 'NroCaracteres', 'Tono',
       'LinkImagen', 'CPE', 'Moneda', 'Audiencia', 'Tema', 'Empresa',
       'NroPagina', 'Dimensión', 'CirculacionMedio', 'AutorConductor',
       'ResumenAclaracion', 'LinkNota', 'Id_Noticia', 'Cuerpo',
       'Link_Original', 'Fecha_Nota'])

In [46]:
# Path To Nokia Unzipped Files (.JSON Format)
unzipped_files='/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1'
lista_zip=glob.glob(unzipped_files+'*.json')
lista_zip

[]

## Read '.JSON' Files Folder

In [47]:
# Path To 'UNZIPPED' Folder - ALL NOKIA FILES
path='/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1'
lista_zip = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.json' in file:
            lista_zip.append(os.path.join(r, file))
lista_zip
print(lista_zip)
                                                                                     
for file in lista_zip:                                                                                    
    data=read_json(file)                                                                                  
    df_json=crear_df_json(data)                                                                           
    print(df_json.shape)                                                                                  
    df_notas=df_notas.append(df_json,'sort=False')                                                        
    print(df_notas.shape)                                                                                 
df_notas.head()                                                                                           
df_notas=df_notas.reset_index(drop=True)
print(df_notas.shape)
df_notas.head()

['/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/ZIP FILES/NokiaFile-20210717121628964.json', '/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/ZIP FILES/NokiaFile-20210630121552169.json', '/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/ZIP FILES/NokiaFile-20210706121531916.json', '/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/ZIP FILES/NokiaFile-20210714121538625.json', '/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/home/roman/Desktop/Porter Novelli MX /PORTER

,IdNoticia,Fecha,Hora,TipoDeMedio,Medio,País,Programa - Sección,Título,Tier,NroCaracteres,Tono,LinkImagen,CPE,Moneda,Audiencia,Tema,Empresa,NroPagina,Dimensión,CirculacionMedio,AutorConductor,ResumenAclaracion,LinkNota,Id_Noticia,Cuerpo,Link_Original,Fecha_Nota,index,Sección,IdMedio,Cpe,IdTema
0,310753252,7/17/2021,00:00:00,Revista,Apertura,Argentina,NaN,Oro Blanco Litio,1,28907,Sin Clasificar,http://news.globalnews.com.ar/Validar.aspx?n=3...,NaN,Peso Argentino,28548,,Dispositivos Nokia -PN,1,4912.0,4758,Tomás Rodríguez Ansorena,La Argentina tiene la oportunidad histórica d...,,NaN,La Argentina tiene la oportunidad histórica de...,NaN,NaN,0.0,Portada,0,1605898.5,None
1,310757610,7/15/2021,00:00:00,Online,Qué onda gye! (Online),Ecuador,NaN,Cinco tips para disfrutar de plena salud con S...,3,3473,Sin Clasificar,http://news.globalnews.com.ar/Validar.aspx?n=3...,NaN,Dólar Estadounidense,166,,Dispositivos Nokia -PN,0,0.0,N/A,,La tecnología puede ser un gran aliado para q...,https://queondagye.com/cinco-tips-para-disfrut...,NaN,La tecnología puede ser un gran aliado para qu...,NaN,NaN,0.0,Gral,0,1215.549927,None
2,310759119,7/15/2021,00:00:00,Online,El Metropolitano Digital (Online),El Salvador,NaN,La tecnología de Samsung derriba mitos de la r...,3,3972,Sin Clasificar,http://news.globalnews.com.ar/Validar.aspx?n=3...,NaN,Dólar Estadounidense,100,,Dispositivos Nokia -PN,0,0.0,N/A,,Solemos centrar la tarea de escoger un refrig...,https://www.elmetropolitanodigital.com/2021/07...,NaN,Solemos centrar la tarea de escoger un refrige...,NaN,NaN,0.0,Gral,0,278.040009,None
3,310759418,7/15/2021,00:00:00,Online,360 El Salvador (Online),El Salvador,NaN,"Xiaomi becomes second-largest smartphone OEM, ...",3,1635,Sin Clasificar,http://news.globalnews.com.ar/Validar.aspx?n=3...,NaN,Dólar Estadounidense,2336,,Dispositivos Nokia -PN,0,0.0,N/A,,"TL;DRAccording to Canalys, Xiaomi is now the ...",https://360elsalvador.com/tecnologia/xiaomi-be...,NaN,"TL;DR<br>According to Canalys, Xiaomi is now t...",NaN,NaN,0.0,Gral,0,114.449997,None
4,310759427,7/16/2021,00:00:00,Online,360 El Salvador (Online),El Salvador,NaN,Second HarmonyOS 2.0 closed beta rolling out f...,3,2112,Sin Clasificar,http://news.globalnews.com.ar/Validar.aspx?n=3...,NaN,Dólar Estadounidense,2336,,Dispositivos Nokia -PN,0,0.0,N/A,,"Recently, Huawei has released a second Harmon...",https://360elsalvador.com/tecnologia/second-ha...,NaN,"Recently, Huawei has released a second Harmony...",NaN,NaN,0.0,Gral,0,147.839996,None


In [48]:
df_notas['CPE'].dtypes

dtype('O')

## Define 'Exchange Rate' to USA Dollars

In [49]:
def cambio_moneda_x(mor,cpe):
    if mor=='Peso Argentino':
        return format(cpe/ARS,'.3f')
    elif mor=='Peso Uruguayo':
        return format(cpe/UYU,'.3f')
    elif mor=='Peso Mejicano':
        return format(cpe/MXN,'.3f')
    elif mor=='Peso Colombiano':
        return format(cpe/COP,'.3f')
    elif mor=='Nuevo Sol Peruano':
        return format(cpe/PEN,'.3f')
    elif mor=='Guaraní Paraguayo':
        return format(cpe/PYG,'.3f')
    elif mor=='Peso Chileno':
        return format(cpe/CLP,'.3f')
    elif mor=='Lempira Hondureña':
        return format(cpe/HNL,'.3f')
    elif mor=='Dólar Estadounidense':
        return format(cpe/1,'.3f')
    elif mor=='Quetzal Guatemalteco':
        return format(cpe/GTQ,'.3f')
    elif mor=='Boliviano':
        return format(cpe/BOB,'.3f')
    elif mor=='Colón Costarricense':
        return format(cpe/CRC,'.3f')
    else: return cpe
cambio_moneda_x('Peso Argentino',1.2)

'0.012'

In [50]:
# Convert CPE Initial Value To USD Dollars
df_notas['CPE_USD'] = df_notas['CPE'].apply(lambda row: cambio_moneda(row.Moneda, row.CPE), axis=1)
print(df_notas.shape)
df_notas.head(5)

TypeError: <lambda>() got an unexpected keyword argument 'axis'

In [51]:
df_notas.columns

Index(['IdNoticia', 'Fecha', 'Hora', 'TipoDeMedio', 'Medio', 'País',
       'Programa - Sección', 'Título', 'Tier', 'NroCaracteres', 'Tono',
       'LinkImagen', 'CPE', 'Moneda', 'Audiencia', 'Tema', 'Empresa',
       'NroPagina', 'Dimensión', 'CirculacionMedio', 'AutorConductor',
       'ResumenAclaracion', 'LinkNota', 'Id_Noticia', 'Cuerpo',
       'Link_Original', 'Fecha_Nota', 'index', 'Sección', 'IdMedio', 'Cpe',
       'IdTema'],
      dtype='object')

In [52]:
df_notas['LinkNota']

0                                                        
1       https://queondagye.com/cinco-tips-para-disfrut...
2       https://www.elmetropolitanodigital.com/2021/07...
3       https://360elsalvador.com/tecnologia/xiaomi-be...
4       https://360elsalvador.com/tecnologia/second-ha...
                              ...                        
6795    https://boyaca.extra.com.co/noticias/ciencia/n...
6796    https://barrancabermeja.extra.com.co/noticias/...
6797    http://www.xataka.com.mx/xataka-mexico/rom-146...
6798    https://www.msn.com/es-mx/dinero/noticias/los-...
6799    https://www.larepublica.co/globoeconomia/se-di...
Name: LinkNota, Length: 6800, dtype: object

In [53]:
df_notas['Moneda'].unique()

array(['Peso Argentino', 'Dólar Estadounidense', 'Peso Mejicano',
       'Nuevo Sol Peruano', 'Peso Colombiano', 'Colón Costarricense',
       'Peso Uruguayo', 'Peso Chileno', 'Guaraní Paraguayo',
       'Quetzal Guatemalteco', 'Lempira Hondureña', 'Boliviano'],
      dtype=object)

## Verify 'df_notas' DataFrame Shape

In [54]:
print(BOLD+'Tamaño de df_notas:'+END+'{}, ({} filas y {} columnas)'.format(df_notas.shape,df_notas.shape[0],df_notas.shape[1]))
print('\n')

print(BOLD+'Nombres de las columnas en df_notas:'+END+'{}'.format(df_notas.columns))

Tamaño de df_notas:(6800, 32), (6800 filas y 32 columnas)


Nombres de las columnas en df_notas:Index(['IdNoticia', 'Fecha', 'Hora', 'TipoDeMedio', 'Medio', 'País',
       'Programa - Sección', 'Título', 'Tier', 'NroCaracteres', 'Tono',
       'LinkImagen', 'CPE', 'Moneda', 'Audiencia', 'Tema', 'Empresa',
       'NroPagina', 'Dimensión', 'CirculacionMedio', 'AutorConductor',
       'ResumenAclaracion', 'LinkNota', 'Id_Noticia', 'Cuerpo',
       'Link_Original', 'Fecha_Nota', 'index', 'Sección', 'IdMedio', 'Cpe',
       'IdTema'],
      dtype='object')


In [55]:
ANALIZED['No']=df_notas['IdNoticia']
ANALIZED['Source']=df_notas['TipoDeMedio']
ANALIZED['Host']=df_notas['Medio']
ANALIZED['Clean_Host']=0
ANALIZED['Link']=df_notas['LinkNota']
ANALIZED['Link Image']=df_notas['LinkImagen']
ANALIZED['CPE']=df_notas['Cpe']
ANALIZED['Moneda']=df_notas['Moneda']
ANALIZED['Audiencia']=df_notas['Audiencia']
ANALIZED['CirculacionMedio']=df_notas['CirculacionMedio']
ANALIZED['Date(ET)']=df_notas['Fecha']
ANALIZED['Author Name']=df_notas['AutorConductor']
ANALIZED['Country']=df_notas['País']
ANALIZED['Sub']='No info'
ANALIZED['Location']='No info'
ANALIZED['Sentiment']=''
ANALIZED['Title']=df_notas['Título']
ANALIZED['Highlight']=''
ANALIZED['Contents']=df_notas['Cuerpo']
ANALIZED['Resumen_Categorias']=pd.np.empty((len(ANALIZED), 0)).tolist()
ANALIZED.head(5)

<ipython-input-55-0229e7fcc735>:20: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  ANALIZED['Resumen_Categorias']=pd.np.empty((len(ANALIZED), 0)).tolist()


,No,Source,Host,Link,Link Image,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment,Clean_Host,CPE,Moneda,Audiencia,CirculacionMedio,Location,Sentiment,Highlight
0,310753252,Revista,Apertura,,http://news.globalnews.com.ar/Validar.aspx?n=3...,7/17/2021,NaN,NaN,NaN,NaN,Tomás Rodríguez Ansorena,NaN,Argentina,No info,Oro Blanco Litio,La Argentina tiene la oportunidad histórica de...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1605898.5,Peso Argentino,28548,4758,No info,,
1,310757610,Online,Qué onda gye! (Online),https://queondagye.com/cinco-tips-para-disfrut...,http://news.globalnews.com.ar/Validar.aspx?n=3...,7/15/2021,NaN,NaN,NaN,NaN,,NaN,Ecuador,No info,Cinco tips para disfrutar de plena salud con S...,La tecnología puede ser un gran aliado para qu...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1215.549927,Dólar Estadounidense,166,N/A,No info,,
2,310759119,Online,El Metropolitano Digital (Online),https://www.elmetropolitanodigital.com/2021/07...,http://news.globalnews.com.ar/Validar.aspx?n=3...,7/15/2021,NaN,NaN,NaN,NaN,,NaN,El Salvador,No info,La tecnología de Samsung derriba mitos de la r...,Solemos centrar la tarea de escoger un refrige...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,278.040009,Dólar Estadounidense,100,N/A,No info,,
3,310759418,Online,360 El Salvador (Online),https://360elsalvador.com/tecnologia/xiaomi-be...,http://news.globalnews.com.ar/Validar.aspx?n=3...,7/15/2021,NaN,NaN,NaN,NaN,,NaN,El Salvador,No info,"Xiaomi becomes second-largest smartphone OEM, ...","TL;DR<br>According to Canalys, Xiaomi is now t...",NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,114.449997,Dólar Estadounidense,2336,N/A,No info,,
4,310759427,Online,360 El Salvador (Online),https://360elsalvador.com/tecnologia/second-ha...,http://news.globalnews.com.ar/Validar.aspx?n=3...,7/16/2021,NaN,NaN,NaN,NaN,,NaN,El Salvador,No info,Second HarmonyOS 2.0 closed beta rolling out f...,"Recently, Huawei has released a second Harmony...",NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,147.839996,Dólar Estadounidense,2336,N/A,No info,,


In [56]:
df_notas['CPE'].unique()

array([nan], dtype=object)

## Verify Time Stamp Format

In [57]:
pd.to_datetime(ANALIZED['Date(ET)'][0],format='%m/%d/%Y')
ANALIZED['Date(ET)']=pd.to_datetime(ANALIZED['Date(ET)'],format='%m/%d/%Y')
ANALIZED.head(3)

,No,Source,Host,Link,Link Image,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment,Clean_Host,CPE,Moneda,Audiencia,CirculacionMedio,Location,Sentiment,Highlight
0,310753252,Revista,Apertura,,http://news.globalnews.com.ar/Validar.aspx?n=3...,2021-07-17,NaN,NaN,NaN,NaN,Tomás Rodríguez Ansorena,NaN,Argentina,No info,Oro Blanco Litio,La Argentina tiene la oportunidad histórica de...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1605898.5,Peso Argentino,28548,4758,No info,,
1,310757610,Online,Qué onda gye! (Online),https://queondagye.com/cinco-tips-para-disfrut...,http://news.globalnews.com.ar/Validar.aspx?n=3...,2021-07-15,NaN,NaN,NaN,NaN,,NaN,Ecuador,No info,Cinco tips para disfrutar de plena salud con S...,La tecnología puede ser un gran aliado para qu...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1215.549927,Dólar Estadounidense,166,N/A,No info,,
2,310759119,Online,El Metropolitano Digital (Online),https://www.elmetropolitanodigital.com/2021/07...,http://news.globalnews.com.ar/Validar.aspx?n=3...,2021-07-15,NaN,NaN,NaN,NaN,,NaN,El Salvador,No info,La tecnología de Samsung derriba mitos de la r...,Solemos centrar la tarea de escoger un refrige...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,278.040009,Dólar Estadounidense,100,N/A,No info,,


## Apply Country, Subsidiary, Q, H, Month & CleanHost Functions

In [58]:
ANALIZED['Date(ET)']=pd.to_datetime(ANALIZED['Date(ET)'], format='%m/%d/%Y')
ANALIZED['Month'] = pd.DatetimeIndex(ANALIZED['Date(ET)']).month
ANALIZED['Country']=ANALIZED['Country'].fillna('MX')
ANALIZED['Country']=ANALIZED['Country'].apply(lambda row : country_sub(row)[0] )
ANALIZED['Sub']=ANALIZED['Country'].apply(lambda row : country_sub(row)[1])
ANALIZED['Full Month']=ANALIZED['Month'].apply(lambda row : find_quarters(str(row))[0])
ANALIZED['Q']=ANALIZED['Month'].apply(lambda row : find_quarters(str(row))[2] )
ANALIZED['H']=ANALIZED['Month'].apply(lambda row : find_quarters(str(row))[3] )
ANALIZED['Month']=ANALIZED['Month'].apply(lambda row : find_quarters(str(row))[1] )
ANALIZED['Link']=ANALIZED['Link'].apply(lambda row:str(row).replace('|HTTP|','http://').replace('|HTTPS|','https://').replace('|WWW|','www'))
ANALIZED['Clean_Host'] = ANALIZED['Link'].str.extract(r'(http[s]?:\/\/.+?\/)')
ANALIZED['Clean_Host']=ANALIZED['Link'].apply(lambda row : str(row).replace('http://','').replace('https://','').split('/', 1)[0])#.replace('',''))
print(ANALIZED.shape)
ANALIZED.head(3)

(6800, 39)


,No,Source,Host,Link,Link Image,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment,Clean_Host,CPE,Moneda,Audiencia,CirculacionMedio,Location,Sentiment,Highlight
0,310753252,Revista,Apertura,,http://news.globalnews.com.ar/Validar.aspx?n=3...,2021-07-17,July,07,Q3,H2,Tomás Rodríguez Ansorena,NaN,Argentina,Argentina,Oro Blanco Litio,La Argentina tiene la oportunidad histórica de...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1605898.5,Peso Argentino,28548,4758,No info,,
1,310757610,Online,Qué onda gye! (Online),https://queondagye.com/cinco-tips-para-disfrut...,http://news.globalnews.com.ar/Validar.aspx?n=3...,2021-07-15,July,07,Q3,H2,,NaN,Ecuador,Rola,Cinco tips para disfrutar de plena salud con S...,La tecnología puede ser un gran aliado para qu...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,queondagye.com,1215.549927,Dólar Estadounidense,166,N/A,No info,,
2,310759119,Online,El Metropolitano Digital (Online),https://www.elmetropolitanodigital.com/2021/07...,http://news.globalnews.com.ar/Validar.aspx?n=3...,2021-07-15,July,07,Q3,H2,,NaN,El Salvador,Rola,La tecnología de Samsung derriba mitos de la r...,Solemos centrar la tarea de escoger un refrige...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,www.elmetropolitanodigital.com,278.040009,Dólar Estadounidense,100,N/A,No info,,


In [59]:
list(ANALIZED['Full Month'].unique())

['July', 'June']

In [60]:
medialist

,Publication,Host,Tier,Media Type,clean_link_site,MediaList
0,NaN,Ambito,1,General News,www.ambito.com,Yes
1,NaN,CanalAR,2,Technology,www.canal-ar.com.ar,Yes
2,NaN,Clarin,1,General News,www.clarin.com,Yes
3,NaN,Cronista,1,General News,www.cronista.com,Yes
4,NaN,Info Technology,2,Technology,www.infotechnology.com,Yes
...,...,...,...,...,...,...
400,NaN,Negocios Globales,2,Logística,nan,Yes
401,NaN,Nueva Mujer [Publimetro],1,Femenina,nan,Yes
402,NaN,Paula,1,Femenina / Moda / Lifestyle,nan,Yes
403,NaN,SML,3,Moda / Lujo,nan,Yes


## Merge MediaList & ANALIZED DataFrames

In [61]:
print('Shape ANALIZED2:',ANALIZED.shape)
ANALIZED3=ANALIZED[ANALIZED['Link']=='']
print('Shape ANALIZED3:',ANALIZED3.shape)
ANALIZED3=ANALIZED3.merge(medialist,
                  left_on='Host',
                  right_on='Host',
                  #on='Host',         
                  how='left').drop(columns=[ 'Publication', 'clean_link_site'])

ANALIZED4=ANALIZED[ANALIZED['Link']!='']
print('Shape ANALIZED4:',ANALIZED4.shape)

ANALIZED4=ANALIZED4.merge(medialist,
                  left_on='Clean_Host',
                  right_on='clean_link_site',
                  how='left').drop(columns=['Publication',  'clean_link_site' ])

ANALIZED4= ANALIZED4.drop(columns='Host_y')
ANALIZED4=ANALIZED4.rename(columns={'Host_x':'Host'})

print(ANALIZED3.shape)
print(ANALIZED4.shape)
print(ANALIZED3.columns)
print(ANALIZED4.columns)

ANALIZED2=ANALIZED3.append(ANALIZED4)

print('Shape ANALIZED:',ANALIZED.shape)
print('Shape ANALIZED2:',ANALIZED2.shape)

Shape ANALIZED2: (6800, 39)
Shape ANALIZED3: (110, 39)
Shape ANALIZED4: (6690, 39)
(118, 42)
(6788, 42)
Index(['No', 'Source', 'Host', 'Link', 'Link Image', 'Date(ET)', 'Full Month',
       'Month', 'Q', 'H', 'Author Name', 'Language', 'Country', 'Sub', 'Title',
       'Contents', 'Impact', 'Topic', 'Action', 'Category', 'Comments',
       'Resumen_Categorias', 'Mayusculas_Title', 'Mayusculas_Contents',
       'Nokia Sentiment', 'HMD Sentiment', 'Samsung Sentiment',
       'Motorola Sentiment', 'Huawei Sentiment', 'Xiaomi Sentiment',
       'LG Sentiment', 'Clean_Host', 'CPE', 'Moneda', 'Audiencia',
       'CirculacionMedio', 'Location', 'Sentiment', 'Highlight', 'Tier',
       'Media Type', 'MediaList'],
      dtype='object')
Index(['No', 'Source', 'Host', 'Link', 'Link Image', 'Date(ET)', 'Full Month',
       'Month', 'Q', 'H', 'Author Name', 'Language', 'Country', 'Sub', 'Title',
       'Contents', 'Impact', 'Topic', 'Action', 'Category', 'Comments',
       'Resumen_Categorias', 'Ma

In [62]:
medialist.columns

Index(['Publication', 'Host', 'Tier', 'Media Type', 'clean_link_site',
       'MediaList'],
      dtype='object')

In [63]:
print(ANALIZED2.columns)
print(ANALIZED3.columns)
print(ANALIZED4.columns)

Index(['No', 'Source', 'Host', 'Link', 'Link Image', 'Date(ET)', 'Full Month',
       'Month', 'Q', 'H', 'Author Name', 'Language', 'Country', 'Sub', 'Title',
       'Contents', 'Impact', 'Topic', 'Action', 'Category', 'Comments',
       'Resumen_Categorias', 'Mayusculas_Title', 'Mayusculas_Contents',
       'Nokia Sentiment', 'HMD Sentiment', 'Samsung Sentiment',
       'Motorola Sentiment', 'Huawei Sentiment', 'Xiaomi Sentiment',
       'LG Sentiment', 'Clean_Host', 'CPE', 'Moneda', 'Audiencia',
       'CirculacionMedio', 'Location', 'Sentiment', 'Highlight', 'Tier',
       'Media Type', 'MediaList'],
      dtype='object')
Index(['No', 'Source', 'Host', 'Link', 'Link Image', 'Date(ET)', 'Full Month',
       'Month', 'Q', 'H', 'Author Name', 'Language', 'Country', 'Sub', 'Title',
       'Contents', 'Impact', 'Topic', 'Action', 'Category', 'Comments',
       'Resumen_Categorias', 'Mayusculas_Title', 'Mayusculas_Contents',
       'Nokia Sentiment', 'HMD Sentiment', 'Samsung Sentiment',


## Verify Finded Media Outlets

In [64]:
print('No. de Notas que están en Media List: ',ANALIZED2['MediaList'].value_counts() )
print('\n')
print('Distribución de los tipos de Notas en Media List: ')
print(ANALIZED2['Media Type'].value_counts())

No. de Notas que están en Media List:  Yes    797
Name: MediaList, dtype: int64


Distribución de los tipos de Notas en Media List: 
General News       506
Technology         224
Business            52
Interés General     15
Name: Media Type, dtype: int64


In [65]:
ANALIZED2.head()

,No,Source,Host,Link,Link Image,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment,Clean_Host,CPE,Moneda,Audiencia,CirculacionMedio,Location,Sentiment,Highlight,Tier,Media Type,MediaList
0,310753252,Revista,Apertura,,http://news.globalnews.com.ar/Validar.aspx?n=3...,2021-07-17,July,07,Q3,H2,Tomás Rodríguez Ansorena,NaN,Argentina,Argentina,Oro Blanco Litio,La Argentina tiene la oportunidad histórica de...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1605898.5,Peso Argentino,28548,4758,No info,,,2.0,Business,Yes
1,310948729,Diario,La Nación,,http://news.globalnews.com.ar/Validar.aspx?n=3...,2021-07-17,July,07,Q3,H2,,NaN,Argentina,Argentina,Un celular de gama media que apuesta todas sus...,Reseñas<br>El teléfono es más grande que su an...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,761500.75,Peso Argentino,886408,221602,No info,,,NaN,NaN,NaN
2,310959769,Diario,El Observador,,http://news.globalnews.com.ar/Validar.aspx?n=3...,2021-07-17,July,07,Q3,H2,Juan Jose Diaz,NaN,Uruguay,Rola,La vida de Arsenio Luzardo,Desde que lo llevaban a entrenar en bicicleta ...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,302597.5625,Peso Uruguayo,32800,8200,No info,,,2.0,General News,Yes
3,310962074,Diario,El Observador,,http://news.globalnews.com.ar/Validar.aspx?n=3...,2021-07-17,July,07,Q3,H2,,NaN,Uruguay,Rola,Regresa Campus Party con edición digital y gra...,"Campus Party, la mayor experiencia de tecnolog...",NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,45173.699219,Peso Uruguayo,32800,8200,No info,,,2.0,General News,Yes
4,310977124,Diario,El Colombiano,,http://news.globalnews.com.ar/Validar.aspx?n=3...,2021-07-17,July,07,Q3,H2,Por JUAN GUILLERMO MORENO,NaN,Colombia,Colombia,Otra crisis en el sector automotor,CARROS INFORME<br>Un problema con<br><br>el su...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,44619648.0,Peso Colombiano,625500,156375,No info,,,2.0,General News,Yes


## Re-Organize Columns

In [66]:
ANALIZED2.columns

Index(['No', 'Source', 'Host', 'Link', 'Link Image', 'Date(ET)', 'Full Month',
       'Month', 'Q', 'H', 'Author Name', 'Language', 'Country', 'Sub', 'Title',
       'Contents', 'Impact', 'Topic', 'Action', 'Category', 'Comments',
       'Resumen_Categorias', 'Mayusculas_Title', 'Mayusculas_Contents',
       'Nokia Sentiment', 'HMD Sentiment', 'Samsung Sentiment',
       'Motorola Sentiment', 'Huawei Sentiment', 'Xiaomi Sentiment',
       'LG Sentiment', 'Clean_Host', 'CPE', 'Moneda', 'Audiencia',
       'CirculacionMedio', 'Location', 'Sentiment', 'Highlight', 'Tier',
       'Media Type', 'MediaList'],
      dtype='object')

In [67]:
ANALIZED=ANALIZED2[['No', 'Source', 'Link', 'Link Image','Host', 'Tier', 'Media Type', 'MediaList', 'Date(ET)', 'Full Month', 'Month',
       'Q', 'H', 'Author Name', 'Language', 'Country', 'Sub', 'Title',
       'Contents', 'Impact', 'Topic', 'Action', 'Category', 'Comments',
       'Resumen_Categorias', 'Mayusculas_Title', 'Mayusculas_Contents',
       'Nokia Sentiment', 'HMD Sentiment', 'Samsung Sentiment',
       'Motorola Sentiment', 'Huawei Sentiment', 'Xiaomi Sentiment',
       'LG Sentiment', 'Clean_Host', 'CPE', #'CPE_USD',
                    'Moneda', 'Audiencia','CirculacionMedio', 'Location', 'Sentiment', 'Highlight',
       #'TAM_CONTENTS'  
]]

ANALIZED=ANALIZED.fillna('')
ANALIZED = ANALIZED.replace('nan','')

print(ANALIZED.shape)
ANALIZED.head()

(6906, 42)


,No,Source,Link,Link Image,Host,Tier,Media Type,MediaList,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment,Clean_Host,CPE,Moneda,Audiencia,CirculacionMedio,Location,Sentiment,Highlight
0,310753252,Revista,,http://news.globalnews.com.ar/Validar.aspx?n=3...,Apertura,2.0,Business,Yes,2021-07-17,July,07,Q3,H2,Tomás Rodríguez Ansorena,,Argentina,Argentina,Oro Blanco Litio,La Argentina tiene la oportunidad histórica de...,,,,,,[],,,,,,,,,,,1.605898e+06,Peso Argentino,28548,4758,No info,,
1,310948729,Diario,,http://news.globalnews.com.ar/Validar.aspx?n=3...,La Nación,,,,2021-07-17,July,07,Q3,H2,,,Argentina,Argentina,Un celular de gama media que apuesta todas sus...,Reseñas<br>El teléfono es más grande que su an...,,,,,,[],,,,,,,,,,,7.615008e+05,Peso Argentino,886408,221602,No info,,
2,310959769,Diario,,http://news.globalnews.com.ar/Validar.aspx?n=3...,El Observador,2.0,General News,Yes,2021-07-17,July,07,Q3,H2,Juan Jose Diaz,,Uruguay,Rola,La vida de Arsenio Luzardo,Desde que lo llevaban a entrenar en bicicleta ...,,,,,,[],,,,,,,,,,,3.025976e+05,Peso Uruguayo,32800,8200,No info,,
3,310962074,Diario,,http://news.globalnews.com.ar/Validar.aspx?n=3...,El Observador,2.0,General News,Yes,2021-07-17,July,07,Q3,H2,,,Uruguay,Rola,Regresa Campus Party con edición digital y gra...,"Campus Party, la mayor experiencia de tecnolog...",,,,,,[],,,,,,,,,,,4.517370e+04,Peso Uruguayo,32800,8200,No info,,
4,310977124,Diario,,http://news.globalnews.com.ar/Validar.aspx?n=3...,El Colombiano,2.0,General News,Yes,2021-07-17,July,07,Q3,H2,Por JUAN GUILLERMO MORENO,,Colombia,Colombia,Otra crisis en el sector automotor,CARROS INFORME<br>Un problema con<br><br>el su...,,,,,,[],,,,,,,,,,,4.461965e+07,Peso Colombiano,625500,156375,No info,,


## Verify Contents Lenght

In [68]:
ANALIZED['Contents']=ANALIZED['Contents'].apply(lambda row: remove_punct(str(row)))
ANALIZED['TAM_CONTENTS']=ANALIZED['Contents'].apply(lambda row: len(str(row)))

print('Notas con mas de 400 caracteres: ', len(ANALIZED[ANALIZED['TAM_CONTENTS']>400]['TAM_CONTENTS']))
print('Notas con menos de 400 caracteres: ', len(ANALIZED[ANALIZED['TAM_CONTENTS']<=400]['TAM_CONTENTS']))

Notas con mas de 400 caracteres:  6830
Notas con menos de 400 caracteres:  76


In [69]:
ANALIZED['Full Content']=''

## Read Nokia Keywords Excel File

In [70]:
df_keywords=pd.read_excel('/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/SOURCE FILES/Nokia_HMD_Keywords (1).xlsx')

dic_keywords=crear_dicc_keywords(df_keywords)
dic_keywords

{'COMPETIDORES': ['Nokia ',
  'Huawei ',
  'Xiaomi ',
  'HMD ',
  'HMD Global ',
  'LG ',
  'Samsung ',
  'Motorola '],
 'HMD Devices': ['Nokia 3310 ',
  'Nokia 2.4',
  'Nokia 2.3 ',
  'Nokia 5.1 Plus ',
  'Nokia 1 ',
  'Nokia 3.1 ',
  'Nokia 8.1 ',
  'Nokia X6 ',
  'Nokia X5 ',
  'Nokia 6.1 ',
  'Nokia 7.1 ',
  'Nokia 5.3 ',
  'Nokia 130 ',
  'Nokia 3310 Dual SIM ',
  'Nokia 3.4',
  'Nokia 1 Plus ',
  'Nokia 216 ',
  'Nokia 230 ',
  'Nokia 5.1 ',
  'Nokia C3 ',
  'Nokia 2.2 ',
  'Nokia C2 ',
  'Nokia 230 Dual SIM ',
  'Nokia 8000 4G',
  'Nokia 3310 3G ',
  'Nokia 1.3 ',
  'Nokia 7 Plus ',
  'Nokia 800 Tough ',
  'Nokia 105 (2017) ',
  'Nokia 150 Dual SIM ',
  'Nokia 106 ',
  'Nokia 210 ',
  'Nokia 105 ',
  'Nokia 5310 ',
  'Nokia 6.1 Plus ',
  'Nokia 2720 Flip ',
  'Nokia 3.1 Plus ',
  'Nokia 2.1 ',
  'Nokia 8110 4G ',
  'Nokia 7.2 ',
  'Nokia 150 ',
  'Nokia C1 ',
  'Nokia 3.2 ',
  'Nokia 4.2 ',
  'Nokia 9 PureView ',
  'Nokia 8 Sirocco ',
  'Nokia 8 ',
  'Nokia 2 ',
  'Nokia 216 Dua

## Fill 'Mayusculas_Contents' & 'Mayusculas_Title' Columns With Uppercase Words

In [71]:
# Llenar las columnas Mayusculas _contents y mayusculas_title
ANALIZED['Mayusculas_Contents']=ANALIZED.apply (lambda row: get_mayusculas(row['Full Content']) if len(row['Full Content'])>=len(row['Contents']) else get_mayusculas(row['Contents']),axis=1)
ANALIZED['Mayusculas_Contents']=ANALIZED['Contents'].apply(lambda row : get_mayusculas(str(row)))
ANALIZED['Mayusculas_Title']=ANALIZED['Title'].apply(lambda row : get_mayusculas(row))
ANALIZED['Mayusculas_Contents'].head(3)

0    [Argentina, Tomás, Rodríguez, Ansorena, Unidos...
1    [Reseñas, Procesador, Snapdragon, Memoria, Alm...
2    [Nacional, Copa, América, Libertadores, Interc...
Name: Mayusculas_Contents, dtype: object

## Create a 'Clean Text' Column

In [72]:
# Hacer columna de texto limpio
ANALIZED['CLEAN_Contents']=ANALIZED['Contents'].apply(lambda row: clean_text_wt(row))
# # Hacer columna de tokens a partir del texto limpio
ANALIZED['1_GRAM_Contents']=ANALIZED['CLEAN_Contents'].apply(lambda row: divide_en_ngramas(row,1))

## Create a New Column From Keywords List (Excel File)

In [73]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
category_list_found = []
for (k, v) in dic_keywords.items():
    k = k.upper()
    ANALIZED[str(k)] = 0
    ANALIZED[str(k) + '_FOUND'] = 0
    renamed = []
    print (k)

    for keyword in v:

        keyword = keyword.lower().strip()
        ANALIZED[k + '_' + keyword] = 0

        if len(keyword.split(' ')) == 1:
            ANALIZED[k + '_' + keyword] = ANALIZED['1_GRAM_Contents'
                    ].apply(lambda row: \
                            find_single_word_in_tokenized_text(row,
                            keyword))
        else:
            ANALIZED[k + '_' + keyword] = ANALIZED['CLEAN_Contents'
                    ].str.contains(str(keyword).lower())

        ANALIZED[k + '_' + keyword] = ANALIZED[k + '_'
                + keyword].apply(lambda x: str(x).replace('True',
                                 keyword))  #
        ANALIZED[k + '_' + keyword] = ANALIZED[k + '_'
                + keyword].apply(lambda x: str(x).replace('False', ''))
        renamed.append(k + '_' + keyword)
    ANALIZED[k] = ANALIZED[renamed].values.tolist()
    ANALIZED[k] = ANALIZED[k].apply(lambda row: list(set(row)))
    ANALIZED[k] = ANALIZED[k].apply(lambda row: [x for x in row
                                    if len(x) > 1])
    ANALIZED[k] = ANALIZED[k].apply(lambda row: ('' if len(row)
                                    < 1 else row))
    ANALIZED[str(k) + '_FOUND'] = ANALIZED[k].apply(lambda row: \
            len(row))
    if k == 'COMPETIDORES' or k == 'HMD DEVICES':
        continue
    else:
        ANALIZED = ANALIZED.drop(columns=renamed)
    category_list_found.append(str(k) + '_FOUND')

ANALIZED['Resumen_Categorias'] = \
    ANALIZED[category_list_found].values.tolist()
ANALIZED['Resumen_Categorias'] = ANALIZED['Resumen_Categorias'
        ].apply(lambda row: convert_to_words(row))
ANALIZED['Resumen_Categorias'].head(3)
ANALIZED = ANALIZED.fillna('')

COMPETIDORES
HMD DEVICES


/home/roman/anaconda3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


SAMSUNG DEVICES
MOTOROLA DEVICES
HUAWEI DEVICES
XIAOMI DEVICES
LG DEVICES
PERSONAS
INICIATIVAS
EVENTOS
RED
GAMA
GAMA ALTA
GAMA MEDIA
GAMA BAJA
REPARACIÓN
PRECIO
OFERTA
POSITIVO
NEGATIVO
TIENDAS
OPERADORES
ACTUALIZACIÍN
OS
TAG AI
TAG TRUST
TAG DATA CENTER
TAG MIXED REALITY
TAG GOVERNMENT
TAG RETAIL
SOCIOS ESTRETÉGICOS 
TECNOLOGÍAS
TAG POLICIA
TAG HEAD MOUNTED DISPLAY
TAG ELECTRONIVOD  
TAG NOKIA NETWORKS


In [74]:
ANALIZED.head(5)

,No,Source,Link,Link Image,Host,Tier,Media Type,MediaList,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment,Clean_Host,CPE,Moneda,Audiencia,CirculacionMedio,Location,Sentiment,Highlight,TAM_CONTENTS,Full Content,CLEAN_Contents,1_GRAM_Contents,COMPETIDORES,COMPETIDORES_FOUND,COMPETIDORES_nokia,COMPETIDORES_huawei,COMPETIDORES_xiaomi,COMPETIDORES_hmd,COMPETIDORES_hmd global,COMPETIDORES_lg,COMPETIDORES_samsung,COMPETIDORES_motorola,HMD DEVICES,HMD DEVICES_FOUND,HMD DEVICES_nokia 3310,HMD DEVICES_nokia 2.4,HMD DEVICES_nokia 2.3,HMD DEVICES_nokia 5.1 plus,HMD DEVICES_nokia 1,HMD DEVICES_nokia 3.1,HMD DEVICES_nokia 8.1,HMD DEVICES_nokia x6,HMD DEVICES_nokia x5,HMD DEVICES_nokia 6.1,HMD DEVICES_nokia 7.1,HMD DEVICES_nokia 5.3,HMD DEVICES_nokia 130,HMD DEVICES_nokia 3310 dual sim,HMD DEVICES_nokia 3.4,HMD DEVICES_nokia 1 plus,HMD DEVICES_nokia 216,HMD DEVICES_nokia 230,HMD DEVICES_nokia 5.1,HMD DEVICES_nokia c3,HMD DEVICES_nokia 2.2,HMD DEVICES_nokia c2,HMD DEVICES_nokia 230 dual sim,HMD DEVICES_nokia 8000 4g,HMD DEVICES_nokia 3310 3g,HMD DEVICES_nokia 1.3,HMD DEVICES_nokia 7 plus,HMD DEVICES_nokia 800 tough,HMD DEVICES_nokia 105 (2017),HMD DEVICES_nokia 150 dual sim,HMD DEVICES_nokia 106,HMD DEVICES_nokia 210,HMD DEVICES_nokia 105,HMD DEVICES_nokia 5310,HMD DEVICES_nokia 6.1 plus,HMD DEVICES_nokia 2720 flip,HMD DEVICES_nokia 3.1 plus,HMD DEVICES_nokia 2.1,HMD DEVICES_nokia 8110 4g,HMD DEVICES_nokia 7.2,HMD DEVICES_nokia 150,HMD DEVICES_nokia c1,HMD DEVICES_nokia 3.2,HMD DEVICES_nokia 4.2,HMD DEVICES_nokia 9 pureview,HMD DEVICES_nokia 8 sirocco,HMD DEVICES_nokia 8,HMD DEVICES_nokia 2,HMD DEVICES_nokia 216 dual sim,HMD DEVICES_nokia 125,HMD DEVICES_nokia 8.3 5g,HMD DEVICES_nokia 8.2,HMD DEVICES_nokia 6300 4g,HMD DEVICES_nokia 6.2,HMD DEVICES_nokia x7,HMD DEVICES_nokia 110,HMD DEVICES_nokia 220 4g,SAMSUNG DEVICES,SAMSUNG DEVICES_FOUND,MOTOROLA DEVICES,MOTOROLA DEVICES_FOUND,HUAWEI DEVICES,HUAWEI DEVICES_FOUND,XIAOMI DEVICES,XIAOMI DEVICES_FOUND,LG DEVICES,LG DEVICES_FOUND,PERSONAS,PERSONAS_FOUND,INICIATIVAS,INICIATIVAS_FOUND,EVENTOS,EVENTOS_FOUND,RED,RED_FOUND,GAMA,GAMA_FOUND,GAMA ALTA,GAMA ALTA_FOUND,GAMA MEDIA,GAMA MEDIA_FOUND,GAMA BAJA,GAMA BAJA_FOUND,REPARACIÓN,REPARACIÓN_FOUND,PRECIO,PRECIO_FOUND,OFERTA,OFERTA_FOUND,POSITIVO,POSITIVO_FOUND,NEGATIVO,NEGATIVO_FOUND,TIENDAS,TIENDAS_FOUND,OPERADORES,OPERADORES_FOUND,ACTUALIZACIÍN,ACTUALIZACIÍN_FOUND,OS,OS_FOUND,TAG AI,TAG AI_FOUND,TAG TRUST,TAG TRUST_FOUND,TAG DATA CENTER,TAG DATA CENTER_FOUND,TAG MIXED REALITY,TAG MIXED REALITY_FOUND,TAG GOVERNMENT,TAG GOVERNMENT_FOUND,TAG RETAIL,TAG RETAIL_FOUND,SOCIOS ESTRETÉGICOS,SOCIOS ESTRETÉGICOS _FOUND,TECNOLOGÍAS,TECNOLOGÍAS_FOUND,TAG POLICIA,TAG POLICIA_FOUND,TAG HEAD MOUNTED DISPLAY,TAG HEAD MOUNTED DISPLAY_FOUND,TAG ELECTRONIVOD,TAG ELECTRONIVOD _FOUND,TAG NOKIA NETWORKS,TAG NOKIA NETWORKS_FOUND
0,310753252,Revista,,http://news.globalnews.com.ar/Validar.aspx?n=3...,Apertura,2.0,Business,Yes,2021-07-17,July,07,Q3,H2,Tomás Rodríguez Ansorena,,Argentina,Argentina,Oro Blanco Litio,La Argentina tiene la oportunidad histórica de...,,,,,,"[LG DEVICES, INICIATIVAS, EVENTOS, RED, GAMA, ...","[Oro, Blanco, Litio]","[Argentina, Tomás, Rodríguez, Ansorena, Unidos...",,,,,,,,,1.605898e+06,Peso Argentino,28548,4758,No info,,,29193,,argentina oportunidad histórica integrar selec...,"[argentina, oportunidad, histórica, integrar, ...","[lg, None]",2,None,None,None,None,,lg,None,None,,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,,0,,0,,0,[None],1,,0,[None],1,[None],1,[None],1,[None],1,[None],1,[None],1,[None],1,"[técnico, None]",2,"[costo, costoso, None]",3,"[oferta, promoción, None]",3,[None],1,[None],1,[None],1,[None],1,[None],1,[None],1,[None],1,[None],1,[None],1,,0,"[ministerio, ministro, None, decreto, l

In [75]:
writer = pd.ExcelWriter(r'NOKIA_JULIO_01-15_2021.xlsx'.format(get_timestampYMD()), 
                        engine='xlsxwriter',options={'strings_to_urls': False})
# [2] Convertir el dataframe en un objeto Excel XlsxWriter.
ANALIZED.to_excel(writer,sheet_name='ANALIZED')#,startrow=1,)

# [3] Definir los objetos workbook y worksheet de xslxwriter.
workbook  = writer.book
worksheet = writer.sheets['ANALIZED']

con_texto = workbook.add_format({'bg_color': '#BCDDE4',
                               #'valign': 'bottom',
                               'align':'center',
#                                'font_name':'Segoe UI Light',
                               'font_color': '#000000'})
general = workbook.add_format({'bold': False,
                               'font_color': 'black',
                               'align': 'vcenter',
                               'align': 'center'})
workbook.formats[0].set_font_size(11)
worksheet.set_row(0, 45)
worksheet.set_column('A:BBB55000', 11)


# [4] Crear el objeto de formato para el encabezado: headercellformat

headercellformat=workbook.add_format({
    'bold': True,
    'text_wrap': True,
#     'font':'Segoe UI Light',
    'font_size':10,
    'font_color':'#FFFFFF',
    'valign': 'vcenter',
    'align': 'center',
#     'width':15,
#     'height':45,
    'bg_color': '#000000',
    'border': 1})

header_format_BIO_ARTICLE= workbook.add_format({
    'bold': True,
    'text_wrap': True,
#     'font':'Segoe UI Light',
    'font_color':'#FFFFFF',
    'valign': 'center',
    'align': 'center',
    'bg_color': '#000000',
    'border': 1})
header_format_ROJO= workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'font_color':'#FFFFFF',
#     'font_name':'Segoe UI Light',
    'valign': 'center',
    'align': 'center',
    'bg_color': '#FF0000',
    'border': 1})
header_format_ANALIZED= workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'font_color':'#FFFFFF',
#     'font_name':'Segoe UI Light',
    'valign': 'center',
    'align': 'center',
    'bg_color': '#4472c4',
    'border': 1})
header_format_CUSTOMER_SOV= workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'font_color':'#FFFFFF',
#     'font_name':'Segoe UI Light',
    'valign': 'center',
    'align': 'center',
    'bg_color': '#C65911',
    'border': 1})

header_format_Morado_Company= workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'font_color':'#FFFFFF',
#     'font_name':'Segoe UI Light',
    'valign': 'center',
    'align': 'center',
    'bg_color': '#7030a0',
    'border': 1})
header_format_Azul_AI_ASSISTANT= workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'font_color':'#000000',
#     'font_name':'Segoe UI Light',
    'valign': 'vcenter',
    'align': 'center',
    'bg_color': '#bdd7ee',
    'border': 1})

#Azul claro: #5b9bd5
#Rojo: #FF0000
#Azul medio: #4472c4
#Rojo fuerte: #83C0C
#Rojo medio:#C65911
#Morado Company: #7030a0
#Azul AI Assistant: #bdd7ee

# [5] Aplicar el formato una columna por vezwrite to the header row **one cell at a time**, with columnname and format
for columnnum, columnname in enumerate(list(ANALIZED.columns)):
    worksheet.write(0, columnnum+1, columnname, headercellformat)


worksheet.conditional_format('Z2:BBB55000', {'type':   'no_blanks',
                                       'format': con_texto})
worksheet.conditional_format('A2:U55000', {'type':   'no_blanks',
                                       'format': general})

worksheet.conditional_format('A1:R1', {'type':   'no_blanks',
                                       'format': header_format_BIO_ARTICLE})
worksheet.conditional_format('S1:U1', {'type':   'no_blanks',
                                       'format': header_format_ROJO})
worksheet.conditional_format('X1:X1', {'type':   'no_blanks',
                                       'format': header_format_ROJO})
worksheet.conditional_format('V1:AZ1', {'type':   'no_blanks',
                                       'format': header_format_ANALIZED})
worksheet.conditional_format('V1:AZ1', {'type':   'no_blanks',
                                       'format': header_format_ANALIZED})
worksheet.conditional_format('BA1:BG1', {'type':   'no_blanks',
                                       'format': header_format_CUSTOMER_SOV})
worksheet.conditional_format('BH1:CC1', {'type':   'no_blanks',
                                       'format': header_format_Morado_Company})
worksheet.conditional_format('CN1:CX1', {'type':   'no_blanks',
                                       'format': header_format_Azul_AI_ASSISTANT})

worksheet.set_column('A:A', None, None, {'hidden': True})


writer.close()

In [76]:
%pwd

'/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Código'

<h1 style="color:orange"> PARTE II </h1>

In [4]:
# Importar librerias
import pandas as pd
import warnings
pd.options.display.max_columns = None
from pandas.io import gbq
pd.set_option('max_rows', 99999)
warnings.filterwarnings("ignore")

In [77]:
# Leer archivo categorizado en - PARTE I
nokia = pd.read_excel('/home/roman/Desktop/Porter Novelli MX /PORTER ETL/Datos/MOMENTO I NOKIA/JULIO FY21 Q1/NOKIA_JULIO_01-15_2021.xlsx')
print(nokia.shape)
nokia.head(5)

(6906, 184)


,Unnamed: 0,No,Source,Link,Link Image,Host,Tier,Media Type,MediaList,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment,Clean_Host,CPE,Moneda,Audiencia,CirculacionMedio,Location,Sentiment,Highlight,TAM_CONTENTS,Full Content,CLEAN_Contents,1_GRAM_Contents,COMPETIDORES,COMPETIDORES_FOUND,COMPETIDORES_nokia,COMPETIDORES_huawei,COMPETIDORES_xiaomi,COMPETIDORES_hmd,COMPETIDORES_hmd global,COMPETIDORES_lg,COMPETIDORES_samsung,COMPETIDORES_motorola,HMD DEVICES,HMD DEVICES_FOUND,HMD DEVICES_nokia 3310,HMD DEVICES_nokia 2.4,HMD DEVICES_nokia 2.3,HMD DEVICES_nokia 5.1 plus,HMD DEVICES_nokia 1,HMD DEVICES_nokia 3.1,HMD DEVICES_nokia 8.1,HMD DEVICES_nokia x6,HMD DEVICES_nokia x5,HMD DEVICES_nokia 6.1,HMD DEVICES_nokia 7.1,HMD DEVICES_nokia 5.3,HMD DEVICES_nokia 130,HMD DEVICES_nokia 3310 dual sim,HMD DEVICES_nokia 3.4,HMD DEVICES_nokia 1 plus,HMD DEVICES_nokia 216,HMD DEVICES_nokia 230,HMD DEVICES_nokia 5.1,HMD DEVICES_nokia c3,HMD DEVICES_nokia 2.2,HMD DEVICES_nokia c2,HMD DEVICES_nokia 230 dual sim,HMD DEVICES_nokia 8000 4g,HMD DEVICES_nokia 3310 3g,HMD DEVICES_nokia 1.3,HMD DEVICES_nokia 7 plus,HMD DEVICES_nokia 800 tough,HMD DEVICES_nokia 105 (2017),HMD DEVICES_nokia 150 dual sim,HMD DEVICES_nokia 106,HMD DEVICES_nokia 210,HMD DEVICES_nokia 105,HMD DEVICES_nokia 5310,HMD DEVICES_nokia 6.1 plus,HMD DEVICES_nokia 2720 flip,HMD DEVICES_nokia 3.1 plus,HMD DEVICES_nokia 2.1,HMD DEVICES_nokia 8110 4g,HMD DEVICES_nokia 7.2,HMD DEVICES_nokia 150,HMD DEVICES_nokia c1,HMD DEVICES_nokia 3.2,HMD DEVICES_nokia 4.2,HMD DEVICES_nokia 9 pureview,HMD DEVICES_nokia 8 sirocco,HMD DEVICES_nokia 8,HMD DEVICES_nokia 2,HMD DEVICES_nokia 216 dual sim,HMD DEVICES_nokia 125,HMD DEVICES_nokia 8.3 5g,HMD DEVICES_nokia 8.2,HMD DEVICES_nokia 6300 4g,HMD DEVICES_nokia 6.2,HMD DEVICES_nokia x7,HMD DEVICES_nokia 110,HMD DEVICES_nokia 220 4g,SAMSUNG DEVICES,SAMSUNG DEVICES_FOUND,MOTOROLA DEVICES,MOTOROLA DEVICES_FOUND,HUAWEI DEVICES,HUAWEI DEVICES_FOUND,XIAOMI DEVICES,XIAOMI DEVICES_FOUND,LG DEVICES,LG DEVICES_FOUND,PERSONAS,PERSONAS_FOUND,INICIATIVAS,INICIATIVAS_FOUND,EVENTOS,EVENTOS_FOUND,RED,RED_FOUND,GAMA,GAMA_FOUND,GAMA ALTA,GAMA ALTA_FOUND,GAMA MEDIA,GAMA MEDIA_FOUND,GAMA BAJA,GAMA BAJA_FOUND,REPARACIÓN,REPARACIÓN_FOUND,PRECIO,PRECIO_FOUND,OFERTA,OFERTA_FOUND,POSITIVO,POSITIVO_FOUND,NEGATIVO,NEGATIVO_FOUND,TIENDAS,TIENDAS_FOUND,OPERADORES,OPERADORES_FOUND,ACTUALIZACIÍN,ACTUALIZACIÍN_FOUND,OS,OS_FOUND,TAG AI,TAG AI_FOUND,TAG TRUST,TAG TRUST_FOUND,TAG DATA CENTER,TAG DATA CENTER_FOUND,TAG MIXED REALITY,TAG MIXED REALITY_FOUND,TAG GOVERNMENT,TAG GOVERNMENT_FOUND,TAG RETAIL,TAG RETAIL_FOUND,SOCIOS ESTRETÉGICOS,SOCIOS ESTRETÉGICOS _FOUND,TECNOLOGÍAS,TECNOLOGÍAS_FOUND,TAG POLICIA,TAG POLICIA_FOUND,TAG HEAD MOUNTED DISPLAY,TAG HEAD MOUNTED DISPLAY_FOUND,TAG ELECTRONIVOD,TAG ELECTRONIVOD _FOUND,TAG NOKIA NETWORKS,TAG NOKIA NETWORKS_FOUND
0,0,310753252,Revista,NaN,http://news.globalnews.com.ar/Validar.aspx?n=3...,Apertura,2.0,Business,Yes,2021-07-17,July,7,Q3,H2,Tomás Rodríguez Ansorena,NaN,Argentina,Argentina,Oro Blanco Litio,La Argentina tiene la oportunidad histórica de...,NaN,NaN,NaN,NaN,NaN,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...","['Oro', 'Blanco', 'Litio']","['Argentina', 'Tomás', 'Rodríguez', 'Ansorena'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.605898e+06,Peso Argentino,28548,4758.0,No info,NaN,NaN,29193,NaN,argentina oportunidad histórica integrar selec...,"['argentina', 'oportunidad', 'histórica', 'int...","['lg', 'None']",2,None,None,None,None,NaN,lg,None,None,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,0,['None'],

In [79]:
# Eliminar columna 'Unnamed'
nokia = nokia.loc[:, ~nokia.columns.str.contains('^Unnamed')]
print(nokia.shape)
nokia.head(5)

(6906, 183)


,No,Source,Link,Link Image,Host,Tier,Media Type,MediaList,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment,Clean_Host,CPE,Moneda,Audiencia,CirculacionMedio,Location,Sentiment,Highlight,TAM_CONTENTS,Full Content,CLEAN_Contents,1_GRAM_Contents,COMPETIDORES,COMPETIDORES_FOUND,COMPETIDORES_nokia,COMPETIDORES_huawei,COMPETIDORES_xiaomi,COMPETIDORES_hmd,COMPETIDORES_hmd global,COMPETIDORES_lg,COMPETIDORES_samsung,COMPETIDORES_motorola,HMD DEVICES,HMD DEVICES_FOUND,HMD DEVICES_nokia 3310,HMD DEVICES_nokia 2.4,HMD DEVICES_nokia 2.3,HMD DEVICES_nokia 5.1 plus,HMD DEVICES_nokia 1,HMD DEVICES_nokia 3.1,HMD DEVICES_nokia 8.1,HMD DEVICES_nokia x6,HMD DEVICES_nokia x5,HMD DEVICES_nokia 6.1,HMD DEVICES_nokia 7.1,HMD DEVICES_nokia 5.3,HMD DEVICES_nokia 130,HMD DEVICES_nokia 3310 dual sim,HMD DEVICES_nokia 3.4,HMD DEVICES_nokia 1 plus,HMD DEVICES_nokia 216,HMD DEVICES_nokia 230,HMD DEVICES_nokia 5.1,HMD DEVICES_nokia c3,HMD DEVICES_nokia 2.2,HMD DEVICES_nokia c2,HMD DEVICES_nokia 230 dual sim,HMD DEVICES_nokia 8000 4g,HMD DEVICES_nokia 3310 3g,HMD DEVICES_nokia 1.3,HMD DEVICES_nokia 7 plus,HMD DEVICES_nokia 800 tough,HMD DEVICES_nokia 105 (2017),HMD DEVICES_nokia 150 dual sim,HMD DEVICES_nokia 106,HMD DEVICES_nokia 210,HMD DEVICES_nokia 105,HMD DEVICES_nokia 5310,HMD DEVICES_nokia 6.1 plus,HMD DEVICES_nokia 2720 flip,HMD DEVICES_nokia 3.1 plus,HMD DEVICES_nokia 2.1,HMD DEVICES_nokia 8110 4g,HMD DEVICES_nokia 7.2,HMD DEVICES_nokia 150,HMD DEVICES_nokia c1,HMD DEVICES_nokia 3.2,HMD DEVICES_nokia 4.2,HMD DEVICES_nokia 9 pureview,HMD DEVICES_nokia 8 sirocco,HMD DEVICES_nokia 8,HMD DEVICES_nokia 2,HMD DEVICES_nokia 216 dual sim,HMD DEVICES_nokia 125,HMD DEVICES_nokia 8.3 5g,HMD DEVICES_nokia 8.2,HMD DEVICES_nokia 6300 4g,HMD DEVICES_nokia 6.2,HMD DEVICES_nokia x7,HMD DEVICES_nokia 110,HMD DEVICES_nokia 220 4g,SAMSUNG DEVICES,SAMSUNG DEVICES_FOUND,MOTOROLA DEVICES,MOTOROLA DEVICES_FOUND,HUAWEI DEVICES,HUAWEI DEVICES_FOUND,XIAOMI DEVICES,XIAOMI DEVICES_FOUND,LG DEVICES,LG DEVICES_FOUND,PERSONAS,PERSONAS_FOUND,INICIATIVAS,INICIATIVAS_FOUND,EVENTOS,EVENTOS_FOUND,RED,RED_FOUND,GAMA,GAMA_FOUND,GAMA ALTA,GAMA ALTA_FOUND,GAMA MEDIA,GAMA MEDIA_FOUND,GAMA BAJA,GAMA BAJA_FOUND,REPARACIÓN,REPARACIÓN_FOUND,PRECIO,PRECIO_FOUND,OFERTA,OFERTA_FOUND,POSITIVO,POSITIVO_FOUND,NEGATIVO,NEGATIVO_FOUND,TIENDAS,TIENDAS_FOUND,OPERADORES,OPERADORES_FOUND,ACTUALIZACIÍN,ACTUALIZACIÍN_FOUND,OS,OS_FOUND,TAG AI,TAG AI_FOUND,TAG TRUST,TAG TRUST_FOUND,TAG DATA CENTER,TAG DATA CENTER_FOUND,TAG MIXED REALITY,TAG MIXED REALITY_FOUND,TAG GOVERNMENT,TAG GOVERNMENT_FOUND,TAG RETAIL,TAG RETAIL_FOUND,SOCIOS ESTRETÉGICOS,SOCIOS ESTRETÉGICOS _FOUND,TECNOLOGÍAS,TECNOLOGÍAS_FOUND,TAG POLICIA,TAG POLICIA_FOUND,TAG HEAD MOUNTED DISPLAY,TAG HEAD MOUNTED DISPLAY_FOUND,TAG ELECTRONIVOD,TAG ELECTRONIVOD _FOUND,TAG NOKIA NETWORKS,TAG NOKIA NETWORKS_FOUND
0,310753252,Revista,NaN,http://news.globalnews.com.ar/Validar.aspx?n=3...,Apertura,2.0,Business,Yes,2021-07-17,July,7,Q3,H2,Tomás Rodríguez Ansorena,NaN,Argentina,Argentina,Oro Blanco Litio,La Argentina tiene la oportunidad histórica de...,NaN,NaN,NaN,NaN,NaN,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...","['Oro', 'Blanco', 'Litio']","['Argentina', 'Tomás', 'Rodríguez', 'Ansorena'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.605898e+06,Peso Argentino,28548,4758.0,No info,NaN,NaN,29193,NaN,argentina oportunidad histórica integrar selec...,"['argentina', 'oportunidad', 'histórica', 'int...","['lg', 'None']",2,None,None,None,None,NaN,lg,None,None,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,0,['None'],1,NaN,0,['Non

In [80]:
# Revisar columnas del dataframe 'nokia'
nokia.columns

Index(['No', 'Source', 'Link', 'Link Image', 'Host', 'Tier', 'Media Type',
       'MediaList', 'Date(ET)', 'Full Month',
       ...
       'TECNOLOGÍAS', 'TECNOLOGÍAS_FOUND', 'TAG POLICIA', 'TAG POLICIA_FOUND',
       'TAG HEAD MOUNTED DISPLAY', 'TAG HEAD MOUNTED DISPLAY_FOUND',
       'TAG ELECTRONIVOD  ', 'TAG ELECTRONIVOD  _FOUND', 'TAG NOKIA NETWORKS',
       'TAG NOKIA NETWORKS_FOUND'],
      dtype='object', length=183)

In [83]:
# Filtrar notas de acuerdo a la quincena correspondiente
nokiaQ = nokia[(nokia['Date(ET)'] >= '2021-07-01') & (nokia['Date(ET)'] <= '2021-07-15')]
nokia.reset_index(drop = True, inplace = True)
print(nokiaQ.shape)
nokiaQ.head(5)

(4899, 183)


,No,Source,Link,Link Image,Host,Tier,Media Type,MediaList,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment,Clean_Host,CPE,Moneda,Audiencia,CirculacionMedio,Location,Sentiment,Highlight,TAM_CONTENTS,Full Content,CLEAN_Contents,1_GRAM_Contents,COMPETIDORES,COMPETIDORES_FOUND,COMPETIDORES_nokia,COMPETIDORES_huawei,COMPETIDORES_xiaomi,COMPETIDORES_hmd,COMPETIDORES_hmd global,COMPETIDORES_lg,COMPETIDORES_samsung,COMPETIDORES_motorola,HMD DEVICES,HMD DEVICES_FOUND,HMD DEVICES_nokia 3310,HMD DEVICES_nokia 2.4,HMD DEVICES_nokia 2.3,HMD DEVICES_nokia 5.1 plus,HMD DEVICES_nokia 1,HMD DEVICES_nokia 3.1,HMD DEVICES_nokia 8.1,HMD DEVICES_nokia x6,HMD DEVICES_nokia x5,HMD DEVICES_nokia 6.1,HMD DEVICES_nokia 7.1,HMD DEVICES_nokia 5.3,HMD DEVICES_nokia 130,HMD DEVICES_nokia 3310 dual sim,HMD DEVICES_nokia 3.4,HMD DEVICES_nokia 1 plus,HMD DEVICES_nokia 216,HMD DEVICES_nokia 230,HMD DEVICES_nokia 5.1,HMD DEVICES_nokia c3,HMD DEVICES_nokia 2.2,HMD DEVICES_nokia c2,HMD DEVICES_nokia 230 dual sim,HMD DEVICES_nokia 8000 4g,HMD DEVICES_nokia 3310 3g,HMD DEVICES_nokia 1.3,HMD DEVICES_nokia 7 plus,HMD DEVICES_nokia 800 tough,HMD DEVICES_nokia 105 (2017),HMD DEVICES_nokia 150 dual sim,HMD DEVICES_nokia 106,HMD DEVICES_nokia 210,HMD DEVICES_nokia 105,HMD DEVICES_nokia 5310,HMD DEVICES_nokia 6.1 plus,HMD DEVICES_nokia 2720 flip,HMD DEVICES_nokia 3.1 plus,HMD DEVICES_nokia 2.1,HMD DEVICES_nokia 8110 4g,HMD DEVICES_nokia 7.2,HMD DEVICES_nokia 150,HMD DEVICES_nokia c1,HMD DEVICES_nokia 3.2,HMD DEVICES_nokia 4.2,HMD DEVICES_nokia 9 pureview,HMD DEVICES_nokia 8 sirocco,HMD DEVICES_nokia 8,HMD DEVICES_nokia 2,HMD DEVICES_nokia 216 dual sim,HMD DEVICES_nokia 125,HMD DEVICES_nokia 8.3 5g,HMD DEVICES_nokia 8.2,HMD DEVICES_nokia 6300 4g,HMD DEVICES_nokia 6.2,HMD DEVICES_nokia x7,HMD DEVICES_nokia 110,HMD DEVICES_nokia 220 4g,SAMSUNG DEVICES,SAMSUNG DEVICES_FOUND,MOTOROLA DEVICES,MOTOROLA DEVICES_FOUND,HUAWEI DEVICES,HUAWEI DEVICES_FOUND,XIAOMI DEVICES,XIAOMI DEVICES_FOUND,LG DEVICES,LG DEVICES_FOUND,PERSONAS,PERSONAS_FOUND,INICIATIVAS,INICIATIVAS_FOUND,EVENTOS,EVENTOS_FOUND,RED,RED_FOUND,GAMA,GAMA_FOUND,GAMA ALTA,GAMA ALTA_FOUND,GAMA MEDIA,GAMA MEDIA_FOUND,GAMA BAJA,GAMA BAJA_FOUND,REPARACIÓN,REPARACIÓN_FOUND,PRECIO,PRECIO_FOUND,OFERTA,OFERTA_FOUND,POSITIVO,POSITIVO_FOUND,NEGATIVO,NEGATIVO_FOUND,TIENDAS,TIENDAS_FOUND,OPERADORES,OPERADORES_FOUND,ACTUALIZACIÍN,ACTUALIZACIÍN_FOUND,OS,OS_FOUND,TAG AI,TAG AI_FOUND,TAG TRUST,TAG TRUST_FOUND,TAG DATA CENTER,TAG DATA CENTER_FOUND,TAG MIXED REALITY,TAG MIXED REALITY_FOUND,TAG GOVERNMENT,TAG GOVERNMENT_FOUND,TAG RETAIL,TAG RETAIL_FOUND,SOCIOS ESTRETÉGICOS,SOCIOS ESTRETÉGICOS _FOUND,TECNOLOGÍAS,TECNOLOGÍAS_FOUND,TAG POLICIA,TAG POLICIA_FOUND,TAG HEAD MOUNTED DISPLAY,TAG HEAD MOUNTED DISPLAY_FOUND,TAG ELECTRONIVOD,TAG ELECTRONIVOD _FOUND,TAG NOKIA NETWORKS,TAG NOKIA NETWORKS_FOUND
19,308666502,Interior,NaN,http://news.globalnews.com.ar/Validar.aspx?n=3...,Los Andes [Mendoza],NaN,NaN,NaN,2021-07-06,July,7,Q3,H2,NaN,NaN,Argentina,Argentina,Garbarino retomó el trabajo con aparente norma...,INCERTIDUMBRE El jueves el gremio y represen...,NaN,NaN,NaN,NaN,NaN,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",['Garbarino'],"['Aun', 'Mauricio', 'Videla', 'Garbarino', 'Gr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,478893.437500,Peso Argentino,93220,23305.0,No info,NaN,NaN,5321,NaN,incertidumbre jueves gremio representantes emp...,"['incertidumbre', 'jueves', 'gremio', 'represe...","['samsung', 'None']",2,None,None,None,None,NaN,None,samsung,None,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,0,['None']

In [85]:
# Verificar fechas únicas en la columna 'Date(ET)'
list(nokiaQ['Date(ET)'].unique())

[numpy.datetime64('2021-07-06T00:00:00.000000000'),
 numpy.datetime64('2021-07-05T00:00:00.000000000'),
 numpy.datetime64('2021-07-14T00:00:00.000000000'),
 numpy.datetime64('2021-07-15T00:00:00.000000000'),
 numpy.datetime64('2021-07-02T00:00:00.000000000'),
 numpy.datetime64('2021-07-10T00:00:00.000000000'),
 numpy.datetime64('2021-07-07T00:00:00.000000000'),
 numpy.datetime64('2021-07-08T00:00:00.000000000'),
 numpy.datetime64('2021-07-09T00:00:00.000000000'),
 numpy.datetime64('2021-07-13T00:00:00.000000000'),
 numpy.datetime64('2021-07-12T00:00:00.000000000'),
 numpy.datetime64('2021-07-04T00:00:00.000000000'),
 numpy.datetime64('2021-07-01T00:00:00.000000000'),
 numpy.datetime64('2021-07-11T00:00:00.000000000'),
 numpy.datetime64('2021-07-03T00:00:00.000000000')]

In [86]:
# Verificar países únicos en la columna 'Country'
nokiaQ['Country'].unique()

array(['Argentina', 'Mexico', 'Peru', 'Chile', 'Colombia', 'Uruguay',
       'Ecuador', 'El Salvador', 'Paraguay', 'Guatemala', 'Costa Rica',
       'Bolivia', 'Honduras'], dtype=object)

In [87]:
# Eliminar registros duplicados por el método de 'ID'
nokiaClean = nokiaQ.drop_duplicates(subset=['No','Source','Host','Link'],keep="first")
nokiaClean.reset_index(drop = True, inplace = True)
print(nokiaClean.shape)
nokiaClean.head(5)

(4819, 183)


,No,Source,Link,Link Image,Host,Tier,Media Type,MediaList,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment,Clean_Host,CPE,Moneda,Audiencia,CirculacionMedio,Location,Sentiment,Highlight,TAM_CONTENTS,Full Content,CLEAN_Contents,1_GRAM_Contents,COMPETIDORES,COMPETIDORES_FOUND,COMPETIDORES_nokia,COMPETIDORES_huawei,COMPETIDORES_xiaomi,COMPETIDORES_hmd,COMPETIDORES_hmd global,COMPETIDORES_lg,COMPETIDORES_samsung,COMPETIDORES_motorola,HMD DEVICES,HMD DEVICES_FOUND,HMD DEVICES_nokia 3310,HMD DEVICES_nokia 2.4,HMD DEVICES_nokia 2.3,HMD DEVICES_nokia 5.1 plus,HMD DEVICES_nokia 1,HMD DEVICES_nokia 3.1,HMD DEVICES_nokia 8.1,HMD DEVICES_nokia x6,HMD DEVICES_nokia x5,HMD DEVICES_nokia 6.1,HMD DEVICES_nokia 7.1,HMD DEVICES_nokia 5.3,HMD DEVICES_nokia 130,HMD DEVICES_nokia 3310 dual sim,HMD DEVICES_nokia 3.4,HMD DEVICES_nokia 1 plus,HMD DEVICES_nokia 216,HMD DEVICES_nokia 230,HMD DEVICES_nokia 5.1,HMD DEVICES_nokia c3,HMD DEVICES_nokia 2.2,HMD DEVICES_nokia c2,HMD DEVICES_nokia 230 dual sim,HMD DEVICES_nokia 8000 4g,HMD DEVICES_nokia 3310 3g,HMD DEVICES_nokia 1.3,HMD DEVICES_nokia 7 plus,HMD DEVICES_nokia 800 tough,HMD DEVICES_nokia 105 (2017),HMD DEVICES_nokia 150 dual sim,HMD DEVICES_nokia 106,HMD DEVICES_nokia 210,HMD DEVICES_nokia 105,HMD DEVICES_nokia 5310,HMD DEVICES_nokia 6.1 plus,HMD DEVICES_nokia 2720 flip,HMD DEVICES_nokia 3.1 plus,HMD DEVICES_nokia 2.1,HMD DEVICES_nokia 8110 4g,HMD DEVICES_nokia 7.2,HMD DEVICES_nokia 150,HMD DEVICES_nokia c1,HMD DEVICES_nokia 3.2,HMD DEVICES_nokia 4.2,HMD DEVICES_nokia 9 pureview,HMD DEVICES_nokia 8 sirocco,HMD DEVICES_nokia 8,HMD DEVICES_nokia 2,HMD DEVICES_nokia 216 dual sim,HMD DEVICES_nokia 125,HMD DEVICES_nokia 8.3 5g,HMD DEVICES_nokia 8.2,HMD DEVICES_nokia 6300 4g,HMD DEVICES_nokia 6.2,HMD DEVICES_nokia x7,HMD DEVICES_nokia 110,HMD DEVICES_nokia 220 4g,SAMSUNG DEVICES,SAMSUNG DEVICES_FOUND,MOTOROLA DEVICES,MOTOROLA DEVICES_FOUND,HUAWEI DEVICES,HUAWEI DEVICES_FOUND,XIAOMI DEVICES,XIAOMI DEVICES_FOUND,LG DEVICES,LG DEVICES_FOUND,PERSONAS,PERSONAS_FOUND,INICIATIVAS,INICIATIVAS_FOUND,EVENTOS,EVENTOS_FOUND,RED,RED_FOUND,GAMA,GAMA_FOUND,GAMA ALTA,GAMA ALTA_FOUND,GAMA MEDIA,GAMA MEDIA_FOUND,GAMA BAJA,GAMA BAJA_FOUND,REPARACIÓN,REPARACIÓN_FOUND,PRECIO,PRECIO_FOUND,OFERTA,OFERTA_FOUND,POSITIVO,POSITIVO_FOUND,NEGATIVO,NEGATIVO_FOUND,TIENDAS,TIENDAS_FOUND,OPERADORES,OPERADORES_FOUND,ACTUALIZACIÍN,ACTUALIZACIÍN_FOUND,OS,OS_FOUND,TAG AI,TAG AI_FOUND,TAG TRUST,TAG TRUST_FOUND,TAG DATA CENTER,TAG DATA CENTER_FOUND,TAG MIXED REALITY,TAG MIXED REALITY_FOUND,TAG GOVERNMENT,TAG GOVERNMENT_FOUND,TAG RETAIL,TAG RETAIL_FOUND,SOCIOS ESTRETÉGICOS,SOCIOS ESTRETÉGICOS _FOUND,TECNOLOGÍAS,TECNOLOGÍAS_FOUND,TAG POLICIA,TAG POLICIA_FOUND,TAG HEAD MOUNTED DISPLAY,TAG HEAD MOUNTED DISPLAY_FOUND,TAG ELECTRONIVOD,TAG ELECTRONIVOD _FOUND,TAG NOKIA NETWORKS,TAG NOKIA NETWORKS_FOUND
0,308666502,Interior,NaN,http://news.globalnews.com.ar/Validar.aspx?n=3...,Los Andes [Mendoza],NaN,NaN,NaN,2021-07-06,July,7,Q3,H2,NaN,NaN,Argentina,Argentina,Garbarino retomó el trabajo con aparente norma...,INCERTIDUMBRE El jueves el gremio y represen...,NaN,NaN,NaN,NaN,NaN,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",['Garbarino'],"['Aun', 'Mauricio', 'Videla', 'Garbarino', 'Gr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,478893.437500,Peso Argentino,93220,23305.0,No info,NaN,NaN,5321,NaN,incertidumbre jueves gremio representantes emp...,"['incertidumbre', 'jueves', 'gremio', 'represe...","['samsung', 'None']",2,None,None,None,None,NaN,None,samsung,None,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,0,['None'],

In [88]:
# Filtrar sólo notas relacionadas con 'Nokia'
nokiaClean = nokiaClean[nokiaClean['COMPETIDORES_nokia'] == 'nokia']
print(nokiaClean.shape)
nokiaClean.head(5)

(554, 183)


,No,Source,Link,Link Image,Host,Tier,Media Type,MediaList,Date(ET),Full Month,Month,Q,H,Author Name,Language,Country,Sub,Title,Contents,Impact,Topic,Action,Category,Comments,Resumen_Categorias,Mayusculas_Title,Mayusculas_Contents,Nokia Sentiment,HMD Sentiment,Samsung Sentiment,Motorola Sentiment,Huawei Sentiment,Xiaomi Sentiment,LG Sentiment,Clean_Host,CPE,Moneda,Audiencia,CirculacionMedio,Location,Sentiment,Highlight,TAM_CONTENTS,Full Content,CLEAN_Contents,1_GRAM_Contents,COMPETIDORES,COMPETIDORES_FOUND,COMPETIDORES_nokia,COMPETIDORES_huawei,COMPETIDORES_xiaomi,COMPETIDORES_hmd,COMPETIDORES_hmd global,COMPETIDORES_lg,COMPETIDORES_samsung,COMPETIDORES_motorola,HMD DEVICES,HMD DEVICES_FOUND,HMD DEVICES_nokia 3310,HMD DEVICES_nokia 2.4,HMD DEVICES_nokia 2.3,HMD DEVICES_nokia 5.1 plus,HMD DEVICES_nokia 1,HMD DEVICES_nokia 3.1,HMD DEVICES_nokia 8.1,HMD DEVICES_nokia x6,HMD DEVICES_nokia x5,HMD DEVICES_nokia 6.1,HMD DEVICES_nokia 7.1,HMD DEVICES_nokia 5.3,HMD DEVICES_nokia 130,HMD DEVICES_nokia 3310 dual sim,HMD DEVICES_nokia 3.4,HMD DEVICES_nokia 1 plus,HMD DEVICES_nokia 216,HMD DEVICES_nokia 230,HMD DEVICES_nokia 5.1,HMD DEVICES_nokia c3,HMD DEVICES_nokia 2.2,HMD DEVICES_nokia c2,HMD DEVICES_nokia 230 dual sim,HMD DEVICES_nokia 8000 4g,HMD DEVICES_nokia 3310 3g,HMD DEVICES_nokia 1.3,HMD DEVICES_nokia 7 plus,HMD DEVICES_nokia 800 tough,HMD DEVICES_nokia 105 (2017),HMD DEVICES_nokia 150 dual sim,HMD DEVICES_nokia 106,HMD DEVICES_nokia 210,HMD DEVICES_nokia 105,HMD DEVICES_nokia 5310,HMD DEVICES_nokia 6.1 plus,HMD DEVICES_nokia 2720 flip,HMD DEVICES_nokia 3.1 plus,HMD DEVICES_nokia 2.1,HMD DEVICES_nokia 8110 4g,HMD DEVICES_nokia 7.2,HMD DEVICES_nokia 150,HMD DEVICES_nokia c1,HMD DEVICES_nokia 3.2,HMD DEVICES_nokia 4.2,HMD DEVICES_nokia 9 pureview,HMD DEVICES_nokia 8 sirocco,HMD DEVICES_nokia 8,HMD DEVICES_nokia 2,HMD DEVICES_nokia 216 dual sim,HMD DEVICES_nokia 125,HMD DEVICES_nokia 8.3 5g,HMD DEVICES_nokia 8.2,HMD DEVICES_nokia 6300 4g,HMD DEVICES_nokia 6.2,HMD DEVICES_nokia x7,HMD DEVICES_nokia 110,HMD DEVICES_nokia 220 4g,SAMSUNG DEVICES,SAMSUNG DEVICES_FOUND,MOTOROLA DEVICES,MOTOROLA DEVICES_FOUND,HUAWEI DEVICES,HUAWEI DEVICES_FOUND,XIAOMI DEVICES,XIAOMI DEVICES_FOUND,LG DEVICES,LG DEVICES_FOUND,PERSONAS,PERSONAS_FOUND,INICIATIVAS,INICIATIVAS_FOUND,EVENTOS,EVENTOS_FOUND,RED,RED_FOUND,GAMA,GAMA_FOUND,GAMA ALTA,GAMA ALTA_FOUND,GAMA MEDIA,GAMA MEDIA_FOUND,GAMA BAJA,GAMA BAJA_FOUND,REPARACIÓN,REPARACIÓN_FOUND,PRECIO,PRECIO_FOUND,OFERTA,OFERTA_FOUND,POSITIVO,POSITIVO_FOUND,NEGATIVO,NEGATIVO_FOUND,TIENDAS,TIENDAS_FOUND,OPERADORES,OPERADORES_FOUND,ACTUALIZACIÍN,ACTUALIZACIÍN_FOUND,OS,OS_FOUND,TAG AI,TAG AI_FOUND,TAG TRUST,TAG TRUST_FOUND,TAG DATA CENTER,TAG DATA CENTER_FOUND,TAG MIXED REALITY,TAG MIXED REALITY_FOUND,TAG GOVERNMENT,TAG GOVERNMENT_FOUND,TAG RETAIL,TAG RETAIL_FOUND,SOCIOS ESTRETÉGICOS,SOCIOS ESTRETÉGICOS _FOUND,TECNOLOGÍAS,TECNOLOGÍAS_FOUND,TAG POLICIA,TAG POLICIA_FOUND,TAG HEAD MOUNTED DISPLAY,TAG HEAD MOUNTED DISPLAY_FOUND,TAG ELECTRONIVOD,TAG ELECTRONIVOD _FOUND,TAG NOKIA NETWORKS,TAG NOKIA NETWORKS_FOUND
6,310278902,Diario,NaN,http://news.globalnews.com.ar/Validar.aspx?n=3...,Las Últimas Noticias,1.0,Interés General,Yes,2021-07-14,July,7,Q3,H2,Francisco Núñez,NaN,Chile,Chile,Ranking: smartphones importados a Chile el pri...,El variopinto panorama de las marcas de smartp...,NaN,NaN,NaN,NaN,NaN,"['SAMSUNG DEVICES', 'XIAOMI DEVICES', 'LG DEVI...","['Ranking', 'Chile']","['Chile', 'Xiaomi', 'Motorola', 'Huawei', 'App...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.159854e+06,Peso Chileno,337760,84440.0,No info,NaN,NaN,6244,NaN,variopinto panorama marcas smartphones venden ...,"['variopinto', 'panorama', 'marcas', 'smartpho...","['samsung', 'xiaomi', 'None', 'lg', 'motorola'...",7,nokia,huawei,xiaomi,None,NaN,lg,samsung,motorola,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [89]:
# Verificar países únicos en la columna 'Country'
nokiaClean['Country'].unique()

array(['Chile', 'Mexico', 'Colombia', 'Argentina', 'Uruguay', 'Peru',
       'El Salvador', 'Guatemala', 'Paraguay', 'Ecuador', 'Bolivia',
       'Costa Rica'], dtype=object)

In [90]:
# Verificar columnas en 'nokiaClean' en formato de lista
for c in nokiaClean.columns:
    print(c)

No
Source
Link
Link Image
Host
Tier
Media Type
MediaList
Date(ET)
Full Month
Month
Q
H
Author Name
Language
Country
Sub
Title
Contents
Impact
Topic
Action
Category
Comments
Resumen_Categorias
Mayusculas_Title
Mayusculas_Contents
Nokia Sentiment
HMD Sentiment
Samsung Sentiment
Motorola Sentiment
Huawei Sentiment
Xiaomi Sentiment
LG Sentiment
Clean_Host
CPE
Moneda
Audiencia
CirculacionMedio
Location
Sentiment
Highlight
TAM_CONTENTS
Full Content
CLEAN_Contents
1_GRAM_Contents
COMPETIDORES
COMPETIDORES_FOUND
COMPETIDORES_nokia
COMPETIDORES_huawei
COMPETIDORES_xiaomi
COMPETIDORES_hmd
COMPETIDORES_hmd global
COMPETIDORES_lg
COMPETIDORES_samsung
COMPETIDORES_motorola
HMD DEVICES
HMD DEVICES_FOUND
HMD DEVICES_nokia 3310
HMD DEVICES_nokia 2.4
HMD DEVICES_nokia 2.3
HMD DEVICES_nokia 5.1 plus
HMD DEVICES_nokia 1
HMD DEVICES_nokia 3.1
HMD DEVICES_nokia 8.1
HMD DEVICES_nokia x6
HMD DEVICES_nokia x5
HMD DEVICES_nokia 6.1
HMD DEVICES_nokia 7.1
HMD DEVICES_nokia 5.3
HMD DEVICES_nokia 130
HMD DEVICES_n

In [91]:
# Obtener subset del dataframe original, para uso específico de los países
nokia = nokiaClean[['No','Source','Host','Country','Title','Tier','Link Image',
                             'CPE','Moneda','Audiencia','Author Name','Link',
                             'Contents','Clean_Host','Date(ET)','Sub','Impact','Topic','Action','Category','Sentiment',
                             'Resumen_Categorias','Month','Full Month','Q','H','TAM_CONTENTS','Mayusculas_Contents',
                             'Mayusculas_Title','CLEAN_Contents','1_GRAM_Contents','COMPETIDORES','COMPETIDORES_FOUND',
                             'COMPETIDORES_xiaomi','COMPETIDORES_lg','COMPETIDORES_hmd global','COMPETIDORES_hmd',
                             'COMPETIDORES_motorola','COMPETIDORES_samsung','COMPETIDORES_huawei','COMPETIDORES_nokia',
                             'TAG POLICIA','TAG POLICIA_FOUND','TAG HEAD MOUNTED DISPLAY','TAG HEAD MOUNTED DISPLAY_FOUND',
                             'TAG ELECTRONIVOD  ','TAG ELECTRONIVOD  _FOUND']]
print(nokia.shape)
nokia.head(5)

(554, 47)


,No,Source,Host,Country,Title,Tier,Link Image,CPE,Moneda,Audiencia,Author Name,Link,Contents,Clean_Host,Date(ET),Sub,Impact,Topic,Action,Category,Sentiment,Resumen_Categorias,Month,Full Month,Q,H,TAM_CONTENTS,Mayusculas_Contents,Mayusculas_Title,CLEAN_Contents,1_GRAM_Contents,COMPETIDORES,COMPETIDORES_FOUND,COMPETIDORES_xiaomi,COMPETIDORES_lg,COMPETIDORES_hmd global,COMPETIDORES_hmd,COMPETIDORES_motorola,COMPETIDORES_samsung,COMPETIDORES_huawei,COMPETIDORES_nokia,TAG POLICIA,TAG POLICIA_FOUND,TAG HEAD MOUNTED DISPLAY,TAG HEAD MOUNTED DISPLAY_FOUND,TAG ELECTRONIVOD,TAG ELECTRONIVOD _FOUND
6,310278902,Diario,Las Últimas Noticias,Chile,Ranking: smartphones importados a Chile el pri...,1.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,3.159854e+06,Peso Chileno,337760,Francisco Núñez,NaN,El variopinto panorama de las marcas de smartp...,NaN,2021-07-14,Chile,NaN,NaN,NaN,NaN,NaN,"['SAMSUNG DEVICES', 'XIAOMI DEVICES', 'LG DEVI...",7,July,Q3,H2,6244,"['Chile', 'Xiaomi', 'Motorola', 'Huawei', 'App...","['Ranking', 'Chile']",variopinto panorama marcas smartphones venden ...,"['variopinto', 'panorama', 'marcas', 'smartpho...","['samsung', 'xiaomi', 'None', 'lg', 'motorola'...",7,xiaomi,lg,NaN,None,motorola,samsung,huawei,nokia,['None'],1,NaN,0,"['audífonos', 'None']",2
7,310307517,Diario,El Economista,Mexico,Altán Redes va a concurso mercantil para asegu...,1.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,7.980000e+04,Peso Mejicano,140000,Nicolás Lucas,NaN,Buscará prórroga con acreedores<br>La empresa ...,NaN,2021-07-14,Mexico,NaN,NaN,NaN,NaN,NaN,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,4193,"['Buscará', 'Altan', 'Redes', 'Red', 'Comparti...","['Altán', 'Redes', 'Red', 'Compartida']",buscará prórroga acreedores empresa informó co...,"['buscará', 'prórroga', 'acreedores', 'empresa...","['nokia', 'huawei', 'None']",3,None,None,NaN,None,None,None,huawei,nokia,['None'],1,NaN,0,['None'],1
15,310535775,Diario,El Colombiano,Colombia,NOVEDADES DE NOKIA QUE LLEGAN AL PAÍS,2.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,6.498459e+06,Peso Colombiano,625500,NaN,NaN,TECNOLOGIA<br><br>Se trata del smartphone Noki...,NaN,2021-07-15,Colombia,NaN,NaN,NaN,NaN,NaN,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,360,"['Nokia', 'Además', 'Nokia', 'Nokia', 'Android...",[],tecnologia trata smartphone nokia x20 conectiv...,"['tecnologia', 'trata', 'smartphone', 'nokia',...","['nokia', 'None']",2,None,None,NaN,None,None,None,None,nokia,['None'],1,NaN,0,['None'],1
16,310551285,Diario,El Financiero,Mexico,"Altán se blinda vs Huawei, Nokia y CDB",1.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,1.590300e+05,Peso Mejicano,367692,Darío Celis,NaN,LTÁN REDES SOLICITÓ ayer formalmente el con A...,NaN,2021-07-15,Mexico,NaN,NaN,NaN,NaN,NaN,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,6871,"['Red', 'Compartida', 'Huawei', 'Nokia', 'Chin...","['Altán', 'Huawei', 'Nokia']",ltán redes solicitó ayer formalmente l1 curso ...,"['ltán', 'redes', 'solicitó', 'ayer', 'formalm...","['nokia', 'huawei', 'None']",3,None,None,NaN,None,None,None,huawei,nokia,['None'],1,NaN,0,['None'],1
23,308452014,Diario,Diario Popular,Argentina,Una campaña vendió más de 11 mil celulares,NaN,http://news.globalnews.com.ar/Validar.aspx?n=3...,4.200772e+05,Peso Argentino,142284,NaN,NaN,POR MAS OE 500 MILLONES DE PES0S<<br>La campa...,NaN,2021-07-05,Argentina,NaN,NaN,NaN,NaN,NaN,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,1856,"['Banco', 'Nación', 'Nativa', 'Mastercard', 'N...",[],oe 500 millones pes0s campaña comprar teléfono...,"['oe', '500', 'millones', 'pes0s', 'campaña', ...","['samsung', 'xiaomi', 'None', 'lg', 'motorola'...",6,xiaomi,lg,NaN,None,motorola,samsung,None,nokia,['None'],1,NaN,0,['None'],1


In [92]:
# Agregar nuevas columnas para control de los datos ingresados por los países
nokia['Ad value2'] = ''
nokia['Audiencia2'] = ''

<ipython-input-92-0bb22aee80f3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nokia['Ad value2'] = ''
<ipython-input-92-0bb22aee80f3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nokia['Audiencia2'] = ''


In [93]:
print(nokia.shape)
nokia.reset_index(drop = True, inplace = True)
nokia.head(3)

(554, 49)


,No,Source,Host,Country,Title,Tier,Link Image,CPE,Moneda,Audiencia,Author Name,Link,Contents,Clean_Host,Date(ET),Sub,Impact,Topic,Action,Category,Sentiment,Resumen_Categorias,Month,Full Month,Q,H,TAM_CONTENTS,Mayusculas_Contents,Mayusculas_Title,CLEAN_Contents,1_GRAM_Contents,COMPETIDORES,COMPETIDORES_FOUND,COMPETIDORES_xiaomi,COMPETIDORES_lg,COMPETIDORES_hmd global,COMPETIDORES_hmd,COMPETIDORES_motorola,COMPETIDORES_samsung,COMPETIDORES_huawei,COMPETIDORES_nokia,TAG POLICIA,TAG POLICIA_FOUND,TAG HEAD MOUNTED DISPLAY,TAG HEAD MOUNTED DISPLAY_FOUND,TAG ELECTRONIVOD,TAG ELECTRONIVOD _FOUND,Ad value2,Audiencia2
0,310278902,Diario,Las Últimas Noticias,Chile,Ranking: smartphones importados a Chile el pri...,1.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,3159854.25,Peso Chileno,337760,Francisco Núñez,NaN,El variopinto panorama de las marcas de smartp...,NaN,2021-07-14,Chile,NaN,NaN,NaN,NaN,NaN,"['SAMSUNG DEVICES', 'XIAOMI DEVICES', 'LG DEVI...",7,July,Q3,H2,6244,"['Chile', 'Xiaomi', 'Motorola', 'Huawei', 'App...","['Ranking', 'Chile']",variopinto panorama marcas smartphones venden ...,"['variopinto', 'panorama', 'marcas', 'smartpho...","['samsung', 'xiaomi', 'None', 'lg', 'motorola'...",7,xiaomi,lg,NaN,None,motorola,samsung,huawei,nokia,['None'],1,NaN,0,"['audífonos', 'None']",2,,
1,310307517,Diario,El Economista,Mexico,Altán Redes va a concurso mercantil para asegu...,1.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,79800.00,Peso Mejicano,140000,Nicolás Lucas,NaN,Buscará prórroga con acreedores<br>La empresa ...,NaN,2021-07-14,Mexico,NaN,NaN,NaN,NaN,NaN,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,4193,"['Buscará', 'Altan', 'Redes', 'Red', 'Comparti...","['Altán', 'Redes', 'Red', 'Compartida']",buscará prórroga acreedores empresa informó co...,"['buscará', 'prórroga', 'acreedores', 'empresa...","['nokia', 'huawei', 'None']",3,None,None,NaN,None,None,None,huawei,nokia,['None'],1,NaN,0,['None'],1,,
2,310535775,Diario,El Colombiano,Colombia,NOVEDADES DE NOKIA QUE LLEGAN AL PAÍS,2.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,6498459.00,Peso Colombiano,625500,NaN,NaN,TECNOLOGIA<br><br>Se trata del smartphone Noki...,NaN,2021-07-15,Colombia,NaN,NaN,NaN,NaN,NaN,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,360,"['Nokia', 'Además', 'Nokia', 'Nokia', 'Android...",[],tecnologia trata smartphone nokia x20 conectiv...,"['tecnologia', 'trata', 'smartphone', 'nokia',...","['nokia', 'None']",2,None,None,NaN,None,None,None,None,nokia,['None'],1,NaN,0,['None'],1,,


In [94]:
# Reordenar columnas para uso específico de la cuenta y países
nokia = nokia[['No','Source','Host','Country','Title','Tier','Link Image',
                             'CPE','Moneda','Audiencia','Author Name','Link',
                             'Contents','Clean_Host','Date(ET)','Sub','Impact','Topic','Action','Category','Sentiment','Ad value2','Audiencia2',
                             'Resumen_Categorias','Month','Full Month','Q','H','TAM_CONTENTS','Mayusculas_Contents',
                             'Mayusculas_Title','CLEAN_Contents','1_GRAM_Contents','COMPETIDORES','COMPETIDORES_FOUND',
                             'COMPETIDORES_xiaomi','COMPETIDORES_lg','COMPETIDORES_hmd global','COMPETIDORES_hmd',
                             'COMPETIDORES_motorola','COMPETIDORES_samsung','COMPETIDORES_huawei','COMPETIDORES_nokia',
                             'TAG POLICIA','TAG POLICIA_FOUND','TAG HEAD MOUNTED DISPLAY','TAG HEAD MOUNTED DISPLAY_FOUND',
                             'TAG ELECTRONIVOD  ','TAG ELECTRONIVOD  _FOUND']]
nokia.reset_index(drop = True, inplace = True)
print(nokia.shape)
nokia.head(5)

(554, 49)


,No,Source,Host,Country,Title,Tier,Link Image,CPE,Moneda,Audiencia,Author Name,Link,Contents,Clean_Host,Date(ET),Sub,Impact,Topic,Action,Category,Sentiment,Ad value2,Audiencia2,Resumen_Categorias,Month,Full Month,Q,H,TAM_CONTENTS,Mayusculas_Contents,Mayusculas_Title,CLEAN_Contents,1_GRAM_Contents,COMPETIDORES,COMPETIDORES_FOUND,COMPETIDORES_xiaomi,COMPETIDORES_lg,COMPETIDORES_hmd global,COMPETIDORES_hmd,COMPETIDORES_motorola,COMPETIDORES_samsung,COMPETIDORES_huawei,COMPETIDORES_nokia,TAG POLICIA,TAG POLICIA_FOUND,TAG HEAD MOUNTED DISPLAY,TAG HEAD MOUNTED DISPLAY_FOUND,TAG ELECTRONIVOD,TAG ELECTRONIVOD _FOUND
0,310278902,Diario,Las Últimas Noticias,Chile,Ranking: smartphones importados a Chile el pri...,1.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,3.159854e+06,Peso Chileno,337760,Francisco Núñez,NaN,El variopinto panorama de las marcas de smartp...,NaN,2021-07-14,Chile,NaN,NaN,NaN,NaN,NaN,,,"['SAMSUNG DEVICES', 'XIAOMI DEVICES', 'LG DEVI...",7,July,Q3,H2,6244,"['Chile', 'Xiaomi', 'Motorola', 'Huawei', 'App...","['Ranking', 'Chile']",variopinto panorama marcas smartphones venden ...,"['variopinto', 'panorama', 'marcas', 'smartpho...","['samsung', 'xiaomi', 'None', 'lg', 'motorola'...",7,xiaomi,lg,NaN,None,motorola,samsung,huawei,nokia,['None'],1,NaN,0,"['audífonos', 'None']",2
1,310307517,Diario,El Economista,Mexico,Altán Redes va a concurso mercantil para asegu...,1.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,7.980000e+04,Peso Mejicano,140000,Nicolás Lucas,NaN,Buscará prórroga con acreedores<br>La empresa ...,NaN,2021-07-14,Mexico,NaN,NaN,NaN,NaN,NaN,,,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,4193,"['Buscará', 'Altan', 'Redes', 'Red', 'Comparti...","['Altán', 'Redes', 'Red', 'Compartida']",buscará prórroga acreedores empresa informó co...,"['buscará', 'prórroga', 'acreedores', 'empresa...","['nokia', 'huawei', 'None']",3,None,None,NaN,None,None,None,huawei,nokia,['None'],1,NaN,0,['None'],1
2,310535775,Diario,El Colombiano,Colombia,NOVEDADES DE NOKIA QUE LLEGAN AL PAÍS,2.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,6.498459e+06,Peso Colombiano,625500,NaN,NaN,TECNOLOGIA<br><br>Se trata del smartphone Noki...,NaN,2021-07-15,Colombia,NaN,NaN,NaN,NaN,NaN,,,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,360,"['Nokia', 'Además', 'Nokia', 'Nokia', 'Android...",[],tecnologia trata smartphone nokia x20 conectiv...,"['tecnologia', 'trata', 'smartphone', 'nokia',...","['nokia', 'None']",2,None,None,NaN,None,None,None,None,nokia,['None'],1,NaN,0,['None'],1
3,310551285,Diario,El Financiero,Mexico,"Altán se blinda vs Huawei, Nokia y CDB",1.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,1.590300e+05,Peso Mejicano,367692,Darío Celis,NaN,LTÁN REDES SOLICITÓ ayer formalmente el con A...,NaN,2021-07-15,Mexico,NaN,NaN,NaN,NaN,NaN,,,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,6871,"['Red', 'Compartida', 'Huawei', 'Nokia', 'Chin...","['Altán', 'Huawei', 'Nokia']",ltán redes solicitó ayer formalmente l1 curso ...,"['ltán', 'redes', 'solicitó', 'ayer', 'formalm...","['nokia', 'huawei', 'None']",3,None,None,NaN,None,None,None,huawei,nokia,['None'],1,NaN,0,['None'],1
4,308452014,Diario,Diario Popular,Argentina,Una campaña vendió más de 11 mil celulares,NaN,http://news.globalnews.com.ar/Validar.aspx?n=3...,4.200772e+05,Peso Argentino,142284,NaN,NaN,POR MAS OE 500 MILLONES DE PES0S<<br>La campa...,NaN,2021-07-05,Argentina,NaN,NaN,NaN,NaN,NaN,,,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,1856,"['Banco', 'Nación', 'Nativa', 'Mastercard', 'N...",[],oe 500 millones pes0s campaña comprar teléfono...,"['oe', '500', 'millones', 'pes0s', 'campaña', ...","['samsung', 'xiaomi', 'None', 'lg', 'motorola'...",6,xiaomi,lg,NaN,None,motorola,samsung,None,nokia,['None'],1,NaN,0,['None'],1


In [96]:
# Verificar países
list(nokia['Country'].unique())

['Chile',
 'Mexico',
 'Colombia',
 'Argentina',
 'Uruguay',
 'Peru',
 'El Salvador',
 'Guatemala',
 'Paraguay',
 'Ecuador',
 'Bolivia',
 'Costa Rica']

In [120]:
# Filtrar y segmentar por países
nokiaPais = nokia[nokia['Country'] == 'Costa Rica']
nokiaPais.reset_index(drop = True, inplace = True)
print(nokiaPais.shape)
nokiaPais.head(3)

(2, 49)


,No,Source,Host,Country,Title,Tier,Link Image,CPE,Moneda,Audiencia,Author Name,Link,Contents,Clean_Host,Date(ET),Sub,Impact,Topic,Action,Category,Sentiment,Ad value2,Audiencia2,Resumen_Categorias,Month,Full Month,Q,H,TAM_CONTENTS,Mayusculas_Contents,Mayusculas_Title,CLEAN_Contents,1_GRAM_Contents,COMPETIDORES,COMPETIDORES_FOUND,COMPETIDORES_xiaomi,COMPETIDORES_lg,COMPETIDORES_hmd global,COMPETIDORES_hmd,COMPETIDORES_motorola,COMPETIDORES_samsung,COMPETIDORES_huawei,COMPETIDORES_nokia,TAG POLICIA,TAG POLICIA_FOUND,TAG HEAD MOUNTED DISPLAY,TAG HEAD MOUNTED DISPLAY_FOUND,TAG ELECTRONIVOD,TAG ELECTRONIVOD _FOUND
0,308891971,Online,La República (Online),Costa Rica,Nokia estaría valorando usar HarmonyOS en uno ...,2.0,http://news.globalnews.com.ar/Validar.aspx?n=3...,429873.750000,Colón Costarricense,58344,Johnny Castro \n[email protected] |\nMiércoles...,https://www.larepublica.net/noticia/nokia-esta...,El nuevo software se encuentra basado en el có...,www.larepublica.net,2021-07-07,Rola,NaN,NaN,NaN,NaN,NaN,,,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,1096,"['Android', 'Servicios', 'Móviles', 'Huawei', ...",['Nokia'],nuevo software encuentra basado código fuente ...,"['nuevo', 'software', 'encuentra', 'basado', '...","['hmd', 'None', 'hmd global', 'nokia', 'huawei']",5,None,None,hmd global,hmd,None,None,huawei,nokia,['None'],1,NaN,0,['None'],1
1,308333931,Online,deporticos.co.cr (Online),Costa Rica,GTA que se ejecuta en un teléfono Android con ...,NaN,http://news.globalnews.com.ar/Validar.aspx?n=3...,89049.765625,Colón Costarricense,2562,Eutropio Llano,https://deporticos.co.cr/gta-que-se-ejecuta-en...,Pero resulta que el tiempo tampoco está muy le...,deporticos.co.cr,2021-07-04,Rola,NaN,NaN,NaN,NaN,NaN,,,"['LG DEVICES', 'INICIATIVAS', 'EVENTOS', 'RED'...",7,July,Q3,H2,2079,"['Android', 'Renegade', 'Project', 'Windows', ...","['Android', 'Windows']",resulta tiempo tampoco lejos gracias truco bás...,"['resulta', 'tiempo', 'tampoco', 'lejos', 'gra...","['nokia', 'None']",2,None,None,NaN,None,None,None,None,nokia,['None'],1,NaN,0,['None'],1


In [121]:
# Guardar en formato Excel (.xlsx)
nokiaPais.to_excel('NOKIA_JULIO_Q1_COSTA_RICA.xlsx')